In [ ]:
# Désinstaller les versions existantes (si nécessaire)
!pip uninstall torch-sparse torch-scatter torch-cluster torch-spline-conv torch-geometric -y
!pip uninstall torch_geometric_temporal -y

# Installer les versions compatibles pour PyTorch 2.5.1 et CUDA 12.1
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install torch-geometric
!pip install torch_geometric_temporal

In [ ]:
# pip install torch-geometric-temporal
import numpy as np
import pandas as pd
import networkx as nx
# import folium
import torch
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error
from torch_geometric_temporal.nn.attention import ASTGCN
from  torch_geometric.loader import TemporalDataLoader
from pytorch_lightning.callbacks import EarlyStopping

In [ ]:
!pip install git+https://github.com/benedekrozemberczki/pytorch_geometric_temporal.git

In [ ]:
import torch
print(torch.__version__)  # Version de PyTorch
print(torch.version.cuda) # Version de CUDA
print(torch.cuda.is_available()) # Vérifie si CUDA est disponible

In [ ]:
import gdown

# Remplace 'file_id' par l'ID réel du fichier Google Drive
file_id = '1_to5cK66dNLtETnEgu-D9FkW2Bu_C5qi'

# Téléchargement du fichier depuis Google Drive
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'M30.csv', quiet=False)


In [ ]:
data = pd.read_csv('M30.csv')
data.head()


In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
data[data['speed_avg'].isna()]

In [ ]:
data.loc[data['speed_avg'].isna() & (data['volume'] == 0), ['speed_avg', 'occupation']] = 0.0




In [ ]:
data.isnull().sum()

In [ ]:
data[data['occupation'].isna()]


In [ ]:
data[data['id'] == 1001]

In [ ]:
data['occupation'] = data.groupby('id')['occupation'].transform(lambda x: x.fillna(x.mean()))
data['speed_avg'] = data.groupby('id')['speed_avg'].transform(lambda x: x.fillna(x.mean()))


In [ ]:
data.isnull().sum()

In [ ]:
data['occupation'].value_counts()

In [ ]:
# Conversion des dates avec utc=True
data['time_bin'] = pd.to_datetime(data['time_bin'], utc=True)

# Extraction des composantes de la date et de l'heure
data['Mois'] = data['time_bin'].dt.month
data['Jour'] = data['time_bin'].dt.day
data['Jour_Semaine'] = data['time_bin'].dt.weekday  # 0 = Lundi, 6 = Dimanche
data['Heure'] = data['time_bin'].dt.hour
data['Minute'] = data['time_bin'].dt.minute







In [ ]:
# Encodage cyclique de l'heure
data['Heure_sin'] = np.sin(2 * np.pi * data['Heure'] / 24)
data['Heure_cos'] = np.cos(2 * np.pi * data['Heure'] / 24)

# Encodage cyclique du jour du mois
data['Jour_sin'] = np.sin(2 * np.pi * data['Jour'] / 31)
data['Jour_cos'] = np.cos(2 * np.pi * data['Jour'] / 31)

# Encodage cyclique du jour de la semaine
data['Jour_Semaine_sin'] = np.sin(2 * np.pi * data['Jour_Semaine'] / 7)
data['Jour_Semaine_cos'] = np.cos(2 * np.pi * data['Jour_Semaine'] / 7)


In [ ]:
data.head()

In [ ]:
data=data.drop('type',axis=1)


In [ ]:
unique_combinations = data[['id','heading', 'lon', 'lat']].drop_duplicates()

In [ ]:
unique_combinations.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt



# Tracer les arcs (en rouge)
plt.figure(figsize=(10, 6))
plt.plot(unique_combinations["lon"], unique_combinations["lat"], linestyle='-', color='red', linewidth=1, label="Arcs")

# Tracer les nœuds (en bleu)
plt.scatter(unique_combinations["lon"], unique_combinations["lat"], color='blue', s=50, label="Nœuds")

# Ajouter des labels et une grille
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Graphique des nœuds et arcs")
plt.legend()
plt.show()


In [ ]:
data['occupation'].value_counts()

In [ ]:
def changer(val):
    if 0.0 <= val < 20.0:
        return "Fluide"
    elif 20.0 <= val < 40.0:
        return "Lent"
    elif 40.0 <= val < 70.0:
        return "Embouteillé"
    else:
        return "blockée"

data["congestion_level"] = data["congestion_level"].apply(changer)


In [ ]:
data['congestion_level'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
EncodLabel=LabelEncoder()
data['congestion_level']=EncodLabel.fit_transform(data['congestion_level'])
for label, encoded in zip(EncodLabel.classes_, range(len(EncodLabel.classes_))):
    print(f"Label: {label}, Encoded Value: {encoded}")


In [ ]:
data.describe()

In [ ]:
import matplotlib.pyplot as plt
value_counts = data['congestion_level'].value_counts()
plt.figure(figsize=(8, 6))
plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Répartition des niveaux de congestion')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Histogramme pour la colonne 'volume'
plt.figure(figsize=(8, 6))
sns.histplot(data['volume'], kde=True, color='blue')
plt.title('Distribution de Volume')
plt.xlabel('Volume')
plt.ylabel('Fréquence')
plt.show()


In [ ]:
# Densité de speed_avg
plt.figure(figsize=(8, 6))
sns.kdeplot(data['speed_avg'], shade=True)
plt.title('Distribution de la vitesse moyenne')
plt.xlabel('Vitesse moyenne ')
plt.ylabel('Densité')
plt.show()

In [ ]:
# Courbe de tendance pour 'volume' par mois
plt.figure(figsize=(10, 6))
sns.lineplot(data=data, x='Mois', y='volume', hue='congestion_level', marker='o')
plt.title('Évolution du Volume par Mois')
plt.xlabel('Mois')
plt.ylabel('Volume')
plt.show()


In [ ]:

# Création des graphiques de tendance entre chaque paire de variables
plt.figure(figsize=(12, 10))

# Tracer volume vs occupation
plt.subplot(2, 2, 1)
sns.regplot(x='volume', y='occupation', data=data, line_kws={"color": "red"})
plt.title('Volume vs Occupation')

# Tracer volume vs congestion_level
plt.subplot(2, 2, 2)
sns.regplot(x='volume', y='congestion_level', data=data, line_kws={"color": "red"})
plt.title('Volume vs Congestion Level')


# Tracer occupation vs congestion_level
plt.subplot(2, 2, 4)
sns.regplot(x='occupation', y='congestion_level', data=data, line_kws={"color": "red"})
plt.title('Occupation vs Congestion Level')

# Afficher la figure
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Sélection des colonnes d'intérêt
columns = ['volume', 'occupation', 'congestion_level','speed_avg']
corr_matrix = data[columns].corr()

# Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matrice de corrélation')
plt.show()

In [ ]:
import folium
from folium.plugins import HeatMap
from IPython.display import display

def afficher_heatmap(data, jour, mois):
    # Filtrer les données pour le jour et le mois spécifiés
    data_filtre = data[(data["Jour"] == jour) & (data["Mois"] == mois)]

    if data_filtre.empty:
        print("Aucune donnée disponible pour cette date.")
        return

    # Créer la carte centrée sur la moyenne des latitudes/longitudes
    m = folium.Map(location=[data_filtre["lat"].mean(), data_filtre["lon"].mean()], zoom_start=12)

    # Ajouter la heatmap
    heat_data = list(zip(data_filtre["lat"], data_filtre["lon"], data_filtre["congestion_level"]))
    HeatMap(heat_data).add_to(m)

    # Afficher la carte directement dans Jupyter
    display(m)


In [ ]:
afficher_heatmap(data, 17, 9)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def afficher_embouteillages_par_heure(data, jour_specifique, mois_specifique):


    df_jour = data[(data["Jour"] == jour_specifique) & (data["Mois"] == mois_specifique)]
    df_jour.groupby("Heure").size().plot(kind="bar", color="red", figsize=(10, 5))
    plt.xlabel("Heure de la journée")
    plt.ylabel("Nombre de véhicule détectés")
    plt.title(f"Répartition des embouteillages par heure ({jour_specifique}/{mois_specifique}/2021)")
    plt.xticks(rotation=0)
    plt.show()




In [ ]:
afficher_embouteillages_par_heure(data, jour_specifique=8, mois_specifique=6)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Choisir les colonnes à normaliser
colonnes_a_normaliser = ['volume', 'occupation', 'congestion_level', 'speed_avg','Mois','Jour','Jour_Semaine','Heure','Minute','Heure_sin','Heure_cos']

# Initialiser le MinMaxScaler
scaler_minmax = MinMaxScaler()

# Appliquer le scaler sur les colonnes sélectionnées
data[colonnes_a_normaliser] = scaler_minmax.fit_transform(data[colonnes_a_normaliser])
data.head()

In [ ]:
# Liste des colonnes à supprimer
delete = ["lat", "lon", "heading","Jour_sin", "Jour_cos", "Jour_Semaine_sin", "Jour_Semaine_cos",]

# Suppression des colonnes
data_final = data.drop(columns=delete,axis=1)

In [ ]:
data_final.columns

In [ ]:
import seaborn as sns
corr_matrix = data_final.corr()

# Affichage avec une heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Corrélation entre les variables et le trafic")
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Créer un index multi-niveaux
time_sensor_index = pd.MultiIndex.from_arrays([data_final['time_bin'].values, data_final['id'].values])
data_final_indexed = data_final.set_index(time_sensor_index)

# Pivoter les données
result = data_final_indexed[['volume','occupation','congestion_level','speed_avg','Mois', 'Jour', 'Jour_Semaine', 'Heure', 'Minute','Heure_sin', 'Heure_cos']].unstack()

# Convertir en tenseur 3D
data_array = result.to_numpy()
num_times = data_array.shape[0]
num_sensors = len(data_final['id'].unique())
num_features = 11
data_restructured_f = data_array.reshape(num_times, num_sensors, num_features)

# Sauvegarder les données
np.save("data_restructured_f.npy", data_restructured_f)
print("Données restructurées sauvegardées ✅")

In [ ]:
# Convertir le tableau numpy en DataFrame
data_restructured_df = pd.DataFrame(data_restructured_f.reshape(-1, data_restructured_f.shape[-1]), columns=['volume','occupation','congestion_level','speed_avg','Mois', 'Jour', 'Jour_Semaine', 'Heure', 'Minute','Heure_sin', 'Heure_cos'])

# Appliquer l'interpolation
data_restructured_interpolated = data_restructured_df.interpolate(method='linear')

# Convertir à nouveau en ndarray si nécessaire
data_restructured_final = data_restructured_interpolated.values.reshape(data_restructured_f.shape)

In [ ]:
# Sauvegarder les données
np.save("data_restructured_f1.npy", data_restructured_final)
print("Données restructurées sauvegardées ✅")

In [ ]:
data_restructured_final.shape

In [ ]:

# Paramètres pour la construction du graphe
DISTANCE_THRESHOLD_KM = 1.0
ADD_SELF_LOOPS = True # Important pour GAT aussi
EDGE_WEIGHT_SIGMA = 0.5 # Conservé pour info, mais GAT apprendra ses propres poids
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    lat1, lat2 = map(math.radians, [lat1, lat2])
    a = math.sin(dLat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dLon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def build_spatial_graph_for_gat(sensor_locations_df, id_col, lat_col, lon_col, distance_threshold_km, add_self_loops=True):
    """
    Construit l'index des arêtes pour GAT (basé sur la proximité).
    Retourne adj_matrix (binaire ou pondérée pour info) et edge_index.
    """
    locations = sensor_locations_df[[id_col, lat_col, lon_col]].drop_duplicates(subset=[id_col]).set_index(id_col)
    sensor_ids = locations.index.tolist()
    num_nodes = len(sensor_ids)
    sensor_id_to_index = {sensor_id: i for i, sensor_id in enumerate(sensor_ids)}

    print(f"Building graph connectivity for {num_nodes} sensors...")
    coords_rad = np.radians(locations[[lat_col, lon_col]].values)
    tree = BallTree(coords_rad, metric='haversine')
    dist_threshold_rad = distance_threshold_km / 6371

    adj_matrix = np.zeros((num_nodes, num_nodes), dtype=np.float32) # Pour info ou debug
    edge_list = []

    # Correction ici: supprimer le '[0]' à la fin car return_distance=False par défaut
    indices = tree.query_radius(coords_rad, r=dist_threshold_rad)

    # Maintenant 'indices' est un tableau où chaque élément est un tableau d'indices de voisins
    for i, neighbors_indices in enumerate(indices):
        # neighbors_indices est maintenant un tableau (itérable)
        for j in neighbors_indices:
            if i == j: continue
            adj_matrix[i, j] = 1 # Matrice binaire simple
            adj_matrix[j, i] = 1 # Symétrique
            # Ajouter la paire (i,j) si elle n'est pas déjà dans edge_list pour éviter doublons GAT
            # (Bien que set() à la fin gère cela aussi)
            if (i,j) not in edge_list and (j,i) not in edge_list:
                 edge_list.append((i, j))


    # Gérer les self-loops pour edge_index si GATConv ne le fait pas (souvent mieux de les inclure)
    if add_self_loops:
        np.fill_diagonal(adj_matrix, 1.0)
        for i in range(num_nodes):
             # Ajouter (i,i) à edge_list si pas déjà présent (pour être sûr)
             if not any(item == (i, i) for item in edge_list):
                 edge_list.append((i, i))

    # Convertir en tensor (set() n'est plus nécessaire si on gère les doublons plus haut)
    # S'assurer qu'il n'y a pas de doublons si on n'a pas géré plus haut
    # edge_list = list(set(edge_list)) # Optionnel si la logique ci-dessus est bonne
   
    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()

    print(f"Graph connectivity built with {edge_index.shape[1]} edges (including self-loops if added).")
    return adj_matrix, edge_index, sensor_id_to_index
adj_matrix2, edge_index2, sensor_id_to_index = build_spatial_graph_for_gat(
    unique_combinations, 'id','lon','lat', DISTANCE_THRESHOLD_KM, ADD_SELF_LOOPS
)

# Move edge_index to the correct device
edge_index3 = edge_index2.to(DEVICE) 
edge_index31 = torch.tensor(edge_index3, dtype=torch.long)



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning import LightningModule, Trainer
# Paramètres
num_timesteps = 12  # Nombre de pas de temps regardés en arrière
num_samples = data_restructured_final.shape[0]

# Séparer 80% Train, 10% Validation, 10% Test
train_size = int(0.8 * num_samples)
val_size = int(0.1 * num_samples)
test_size = num_samples - train_size - val_size


train_data = data_restructured_final[:train_size]
temp_data = data_restructured_final[train_size:]
val_data = temp_data[:val_size]
test_data = temp_data[val_size:]

In [ ]:


from torch.utils.data import Dataset, DataLoader
class TrafficDataset(Dataset):
    def __init__(self, data, num_timesteps=12):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.num_timesteps = num_timesteps

    def __len__(self):
        return len(self.data) - self.num_timesteps

    def __getitem__(self, idx):
        x = self.data[idx : idx + self.num_timesteps]  # (num_timesteps, num_sensors, num_features)
        y = self.data[idx + self.num_timesteps, :, :4]  # Seulement les 4 premières variables (volume, occupation, congestion_level, speed_avg)
        return x.permute(1, 2, 0), y  # ASTGCN attend (num_sensors, num_features, num_timesteps)

# Créer les datasets
train_dataset = TrafficDataset(train_data)
val_dataset = TrafficDataset(val_data)
test_dataset = TrafficDataset(test_data)











In [ ]:
# Créer les DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
edge_index = np.array(np.nonzero(adj_matrix2))  # Convertir la matrice d'adjacence en indices
edge_index31 = torch.tensor(edge_index3, dtype=torch.long)
print(f"✅ DataLoaders prêts : Train={len(train_loader)} batches, Val={len(val_loader)}, Test={len(test_loader)}")
print(edge_index)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import os

# Logger TensorBoard
# from pytorch_lightning.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_loss", patience=7, mode="min")
# logger = TensorBoardLogger("logs", name="astgcn_experiment")
# checkpoint_callback = ModelCheckpoint(
#     dirpath="/content/drive/MyDrive/checkpoints8/",  # Chemin vers Google Drive
#     filename="astgcn-{epoch:02d}-{val_loss:.4f}",
#     save_top_k=3,
#     monitor="val_loss",
#     mode="min",
#     every_n_epochs=1,
# )
# checkpoint_dir = "/content/drive/MyDrive/checkpoints8/"
# if os.path.exists(checkpoint_dir) and os.listdir(checkpoint_dir):
#       latest_checkpoint = checkpoint_dir + max(
#         os.listdir(checkpoint_dir),
#         key=lambda f: int(f.split('epoch=')[1].split('-')[0])  # Extract epoch number
#     )
#       print(f"Dernier checkpoint trouvé : {latest_checkpoint}")
# else:
#     latest_checkpoint = None
#     print("Aucun checkpoint trouvé. Démarrage d'un nouvel entraînement.")

# Configuration du Trainer avec TensorBoard
trainer = Trainer(
    max_epochs=20,  # Nombre maximal d'époques
    callbacks=[early_stopping],  # Callback pour l'early stopping
    log_every_n_steps=1,  # Logger les métriques à chaque étape
    check_val_every_n_epoch=1,  # Valider le modèle à chaque époque
    enable_progress_bar=True,  # Activer la barre de progression
    enable_model_summary=True,  # Afficher un résumé du modèle
    accelerator="auto",  # Utiliser automatiquement le GPU si disponible
    devices="auto",  # Utiliser automatiquement le GPU si disponible


)

In [ ]:
# Convertir adj_matrix1 en torch.Tensor
adj_matrix1_tensor = torch.tensor(adj_matrix1, dtype=torch.float32)

# Générer edge_index à partir de la matrice d'adjacence
edge_index1 = (adj_matrix1_tensor > 0).nonzero(as_tuple=False).t().contiguous()

print("edge_index1 :", edge_index1)

In [ ]:
print("Shape de edge_index:", edge_index31.shape)  # Doit être [2, nombre_d_arêtes]

In [ ]:
# Récupérer un batch
x_batch, y_batch = next(iter(train_loader))

# Afficher la forme du batch
print("Forme du batch x:", x_batch.shape)
print("Nombre de features:", x_batch.shape[-2])

# Si vous voulez afficher les données
print("\nPremiers éléments du batch:")
print(x_batch[0])  # Affiche le premier exemple du batch

In [ ]:
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import torch
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, List

class ASTGCNModel(pl.LightningModule):
    def __init__(self, num_nodes, edge_index, num_features=11, num_timesteps=12, 
                 output_features=4, dropout=0.3, nb_chev_filter=16, nb_time_filter=32):
        super().__init__()
        self.num_nodes = num_nodes
        self.edge_index = edge_index
        self.output_features = output_features
        self.variable_names = ['volume', 'occupation', 'congestion_level', 'speed_avg']

        self.astgcn = ASTGCN(
            nb_block=2,
            in_channels=num_features,
            K=3,
            nb_chev_filter=nb_chev_filter,
            nb_time_filter=nb_time_filter,
            time_strides=1,
            num_for_predict=num_timesteps,
            len_input=num_timesteps,
            num_of_vertices=num_nodes,
        )

        # Projection finale pour obtenir les 4 features
        self.output_layer = nn.Sequential(
            nn.Linear(num_timesteps, 64),
            nn.ReLU(),
            nn.Linear(64, output_features))
            
        self.dropout = nn.Dropout(dropout)
        
        # Stockage des métriques globales
        self.train_metrics = {'loss': [], 'mse': [], 'rmse': [], 'mae': [], 'r2': []}
        self.val_metrics = {'loss': [], 'mse': [], 'rmse': [], 'mae': [], 'r2': []}
        
        # Stockage des métriques par variable
        self.train_var_metrics = {var: {'mse': [], 'rmse': [], 'mae': [], 'r2': []} 
                                 for var in self.variable_names}
        self.val_var_metrics = {var: {'mse': [], 'rmse': [], 'mae': [], 'r2': []} 
                               for var in self.variable_names}

    def forward(self, x):
        x = x.to(self.device)
        self.edge_index = self.edge_index.to(self.device)
        
        # 1. Passage à travers ASTGCN
        # Sortie shape: [batch, nodes, timesteps]
        x = self.astgcn(x, self.edge_index)
        
        # 2. Projection vers les 4 features
        # [batch, nodes, timesteps] -> [batch, nodes, 4]
        x = self.output_layer(x)
        
        return x

    def _compute_metrics(self, y_hat, y):
        """Calcule les métriques globales"""
        y_hat_np = y_hat.detach().cpu().numpy().flatten()
        y_np = y.detach().cpu().numpy().flatten()
        
        mse = mean_squared_error(y_np, y_hat_np)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_np, y_hat_np)
        r2 = r2_score(y_np, y_hat_np)
        
        return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

    def _compute_metrics_by_variable(self, y_hat, y):
        """Calcule les métriques pour chaque variable séparément"""
        metrics = {}
        for i, var_name in enumerate(self.variable_names):
            y_hat_var = y_hat[..., i].flatten().detach().cpu().numpy()
            y_var = y[..., i].flatten().detach().cpu().numpy()
            
            metrics[var_name] = {
                'mse': mean_squared_error(y_var, y_hat_var),
                'rmse': np.sqrt(mean_squared_error(y_var, y_hat_var)),
                'mae': mean_absolute_error(y_var, y_hat_var),
                'r2': r2_score(y_var, y_hat_var)
            }
        return metrics

    def training_step(self, batch, batch_idx):
        x, y = batch
        x, y = x.to(self.device), y.to(self.device)
        y_hat = self(x)
        
        # Calcul des métriques
        loss = nn.MSELoss()(y_hat, y)
        metrics = self._compute_metrics(y_hat, y)
        var_metrics = self._compute_metrics_by_variable(y_hat, y)
        
        # Logging global
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_mse", metrics['mse'], on_step=False, on_epoch=True)
        self.log("train_rmse", metrics['rmse'], on_step=False, on_epoch=True)
        self.log("train_mae", metrics['mae'], on_step=False, on_epoch=True)
        self.log("train_r2", metrics['r2'], on_step=False, on_epoch=True, prog_bar=True)
        
        # Logging par variable
        for var_name in self.variable_names:
            self.log(f"train_{var_name}_mse", var_metrics[var_name]['mse'], on_step=False, on_epoch=True)
            self.log(f"train_{var_name}_rmse", var_metrics[var_name]['rmse'], on_step=False, on_epoch=True)
            self.log(f"train_{var_name}_mae", var_metrics[var_name]['mae'], on_step=False, on_epoch=True)
            self.log(f"train_{var_name}_r2", var_metrics[var_name]['r2'], on_step=False, on_epoch=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x, y = x.to(self.device), y.to(self.device)
        y_hat = self(x)
        
        # Calcul des métriques
        loss = nn.MSELoss()(y_hat, y)
        metrics = self._compute_metrics(y_hat, y)
        var_metrics = self._compute_metrics_by_variable(y_hat, y)
        
        # Logging global
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_mse", metrics['mse'], on_step=False, on_epoch=True)
        self.log("val_rmse", metrics['rmse'], on_step=False, on_epoch=True)
        self.log("val_mae", metrics['mae'], on_step=False, on_epoch=True)
        self.log("val_r2", metrics['r2'], on_step=False, on_epoch=True, prog_bar=True)
        
        # Logging par variable
        for var_name in self.variable_names:
            self.log(f"val_{var_name}_mse", var_metrics[var_name]['mse'], on_step=False, on_epoch=True)
            self.log(f"val_{var_name}_rmse", var_metrics[var_name]['rmse'], on_step=False, on_epoch=True)
            self.log(f"val_{var_name}_mae", var_metrics[var_name]['mae'], on_step=False, on_epoch=True)
            self.log(f"val_{var_name}_r2", var_metrics[var_name]['r2'], on_step=False, on_epoch=True)
        
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        x, y = x.to(self.device), y.to(self.device)
        y_hat = self(x)
        
        # Calcul des métriques
        loss = nn.MSELoss()(y_hat, y)
        metrics = self._compute_metrics(y_hat, y)
        var_metrics = self._compute_metrics_by_variable(y_hat, y)
        
        # Logging global
        self.log("test_loss", loss, on_step=False, on_epoch=True)
        self.log("test_mse", metrics['mse'], on_step=False, on_epoch=True)
        self.log("test_rmse", metrics['rmse'], on_step=False, on_epoch=True)
        self.log("test_mae", metrics['mae'], on_step=False, on_epoch=True)
        self.log("test_r2", metrics['r2'], on_step=False, on_epoch=True)
        
        # Logging par variable
        for var_name in self.variable_names:
            self.log(f"test_{var_name}_mse", var_metrics[var_name]['mse'], on_step=False, on_epoch=True)
            self.log(f"test_{var_name}_rmse", var_metrics[var_name]['rmse'], on_step=False, on_epoch=True)
            self.log(f"test_{var_name}_mae", var_metrics[var_name]['mae'], on_step=False, on_epoch=True)
            self.log(f"test_{var_name}_r2", var_metrics[var_name]['r2'], on_step=False, on_epoch=True)
        
        return {'test_loss': loss, 'metrics': metrics, 'var_metrics': var_metrics}

    def on_train_epoch_end(self):
        # Stockage des métriques globales
        metrics = self.trainer.callback_metrics
        epoch = self.current_epoch
        
        if 'train_loss' in metrics:
            self.train_metrics['loss'].append(metrics['train_loss'].item())
            self.train_metrics['mse'].append(metrics['train_mse'].item())
            self.train_metrics['rmse'].append(metrics['train_rmse'].item())
            self.train_metrics['mae'].append(metrics['train_mae'].item())
            self.train_metrics['r2'].append(metrics['train_r2'].item())
        
        # Stockage des métriques par variable
        for var_name in self.variable_names:
            self.train_var_metrics[var_name]['mse'].append(metrics[f'train_{var_name}_mse'].item())
            self.train_var_metrics[var_name]['rmse'].append(metrics[f'train_{var_name}_rmse'].item())
            self.train_var_metrics[var_name]['mae'].append(metrics[f'train_{var_name}_mae'].item())
            self.train_var_metrics[var_name]['r2'].append(metrics[f'train_{var_name}_r2'].item())
        
        # Affichage
        print(f"\nEpoch {epoch} - Train Metrics:")
        print(f"Global - Loss: {metrics['train_loss'].item():.4f} | MSE: {metrics['train_mse'].item():.4f} | RMSE: {metrics['train_rmse'].item():.4f} | MAE: {metrics['train_mae'].item():.4f} | R²: {metrics['train_r2'].item():.4f}")
        
        for var_name in self.variable_names:
            print(f"{var_name}:")
            print(f"  MSE: {metrics[f'train_{var_name}_mse'].item():.4f} | RMSE: {metrics[f'train_{var_name}_rmse'].item():.4f}")
            print(f"  MAE: {metrics[f'train_{var_name}_mae'].item():.4f} | R²: {metrics[f'train_{var_name}_r2'].item():.4f}")
            print("---")

    def on_validation_epoch_end(self):
        # Stockage des métriques globales
        metrics = self.trainer.callback_metrics
        epoch = self.current_epoch
        
        if 'val_loss' in metrics:
            self.val_metrics['loss'].append(metrics['val_loss'].item())
            self.val_metrics['mse'].append(metrics['val_mse'].item())
            self.val_metrics['rmse'].append(metrics['val_rmse'].item())
            self.val_metrics['mae'].append(metrics['val_mae'].item())
            self.val_metrics['r2'].append(metrics['val_r2'].item())
        
        # Stockage des métriques par variable
        for var_name in self.variable_names:
            self.val_var_metrics[var_name]['mse'].append(metrics[f'val_{var_name}_mse'].item())
            self.val_var_metrics[var_name]['rmse'].append(metrics[f'val_{var_name}_rmse'].item())
            self.val_var_metrics[var_name]['mae'].append(metrics[f'val_{var_name}_mae'].item())
            self.val_var_metrics[var_name]['r2'].append(metrics[f'val_{var_name}_r2'].item())
        
        # Affichage comme dans la capture d'écran
        print(f"\nEpoch {epoch} - Validation Metrics by Variable:")
        for var_name in self.variable_names:
            print(f"{var_name}:")
            print(f"    MSE: {metrics[f'val_{var_name}_mse'].item():.4f}, RMSE: {metrics[f'val_{var_name}_rmse'].item():.4f}")
            print(f"    MAE: {metrics[f'val_{var_name}_mae'].item():.4f}, R²: {metrics[f'val_{var_name}_r2'].item():.4f}")
            print("---")

    def on_test_epoch_end(self):
        metrics = self.trainer.callback_metrics
        
        # Affichage des résultats finaux comme dans la capture d'écran
        print("\nFinal Test Metrics by Variable:")
        for var_name in self.variable_names:
            print(f"{var_name}:")
            print(f"    MSE: {metrics[f'test_{var_name}_mse'].item():.4f}, RMSE: {metrics[f'test_{var_name}_rmse'].item():.4f}")
            print(f"    MAE: {metrics[f'test_{var_name}_mae'].item():.4f}, R²: {metrics[f'test_{var_name}_r2'].item():.4f}")
            print("---")

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=4e-4, weight_decay=1e-6)

    def plot_metrics(self):
        """Trace les courbes d'apprentissage"""
        plt.figure(figsize=(15, 10))
        
        # Plot Loss
        plt.subplot(2, 2, 1)
        plt.plot(self.train_metrics['loss'], 'b-', label="Train Loss")
        plt.plot(self.val_metrics['loss'], 'r-', label="Val Loss")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Training and Validation Loss")
        plt.legend()
        plt.grid()

        # Plot R2
        plt.subplot(2, 2, 2)
        plt.plot(self.train_metrics['r2'], 'b-', label="Train R²")
        plt.plot(self.val_metrics['r2'], 'r-', label="Val R²")
        plt.xlabel("Epoch")
        plt.ylabel("R² Score")
        plt.title("R² Score")
        plt.legend()
        plt.grid()

        # Plot RMSE
        plt.subplot(2, 2, 3)
        plt.plot(self.train_metrics['rmse'], 'b-', label="Train RMSE")
        plt.plot(self.val_metrics['rmse'], 'r-', label="Val RMSE")
        plt.xlabel("Epoch")
        plt.ylabel("RMSE")
        plt.title("RMSE")
        plt.legend()
        plt.grid()

        # Plot MAE
        plt.subplot(2, 2, 4)
        plt.plot(self.train_metrics['mae'], 'b-', label="Train MAE")
        plt.plot(self.val_metrics['mae'], 'r-', label="Val MAE")
        plt.xlabel("Epoch")
        plt.ylabel("MAE")
        plt.title("MAE")
        plt.legend()
        plt.grid()

        plt.tight_layout()
        plt.savefig("training_metrics.png")
        plt.show()

    def plot_variable_metrics(self):
        """Trace les métriques pour chaque variable séparément"""
        fig, axs = plt.subplots(4, 4, figsize=(20, 15))
        
        for i, var_name in enumerate(self.variable_names):
            # MSE
            axs[0, i].plot(self.train_var_metrics[var_name]['mse'], 'b-', label="Train")
            axs[0, i].plot(self.val_var_metrics[var_name]['mse'], 'r-', label="Val")
            axs[0, i].set_title(f"{var_name} - MSE")
            axs[0, i].grid()
            
            # RMSE
            axs[1, i].plot(self.train_var_metrics[var_name]['rmse'], 'b-', label="Train")
            axs[1, i].plot(self.val_var_metrics[var_name]['rmse'], 'r-', label="Val")
            axs[1, i].set_title(f"{var_name} - RMSE")
            axs[1, i].grid()
            
            # MAE
            axs[2, i].plot(self.train_var_metrics[var_name]['mae'], 'b-', label="Train")
            axs[2, i].plot(self.val_var_metrics[var_name]['mae'], 'r-', label="Val")
            axs[2, i].set_title(f"{var_name} - MAE")
            axs[2, i].grid()
            
            # R2
            axs[3, i].plot(self.train_var_metrics[var_name]['r2'], 'b-', label="Train")
            axs[3, i].plot(self.val_var_metrics[var_name]['r2'], 'r-', label="Val")
            axs[3, i].set_title(f"{var_name} - R²")
            axs[3, i].grid()
        
        plt.tight_layout()
        plt.savefig("variable_metrics.png")
        plt.show()

In [ ]:

model5 = ASTGCNModel(
    num_nodes=421,
    num_features=11,
    num_timesteps=12,
    edge_index=edge_index31,  # Assurez-vous que edge_index1 est correctement défini
    dropout=0.4
)

# Définir l'appareil (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de {device}")

# Déplacer le modèle sur l'appareil
model5.to(device)


In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# Add this before initializing your model and trainer

In [ ]:
import pytorch_lightning as pl
trainer = pl.Trainer(max_epochs=20)
trainer.fit(model5, train_loader, val_loader)

In [ ]:
# Test final
results = trainer.test(model5, test_loader)
print(results)

In [ ]:
print("Forme des données d'entraînement:", train_data.shape)
print("Forme des données de validation:", val_data.shape)
print("Forme des données de test:", test_data.shape)

In [ ]:
# Passer l'ensemble de test dans le modèle
model5.eval()  # Mettre le modèle en mode évaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model5.to(device)
# Test loop
with torch.no_grad():
    predictions= []
    true_values = []
    for batch in test_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)  # Déplacer les données sur le même appareil que le modèle
        y_hat = model5(x)
        predictions.append(y_hat)
        true_values.append(y)

# Concaténer les résultats
predictions = torch.cat(predictions, dim=0)
true_values = torch.cat(true_values, dim=0)

# Afficher les résultats
print("Predictions:", predictions)
print("True values:", true_values)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from sklearn.metrics import r2_score

# Définir le jour spécifique à visualiser
specific_day = datetime(2021, 9, 2)  # Par exemple, le 5 mars 2025

# Créer un filtre pour sélectionner uniquement les données de ce jour
# Supposons que vous avez 12 points par jour (toutes les 2 heures)
start_date = datetime(2021, 6, 1)  # Date de début de vos données
dates = [start_date + timedelta(hours=2*i) for i in range(len(true_values))]

# Filtrer les indices correspondant au jour spécifique
day_indices = [i for i, date in enumerate(dates) if date.date() == specific_day.date()]

# Choisir un nœud spécifique
node_idx = 10

# Noms des caractéristiques
feature_names = ["Volume", "Occupation", "Niveau de congestion", "Vitesse moyenne"]

# Créer une figure 2x2 pour les 4 caractéristiques
fig, axs = plt.subplots(2, 2, figsize=(15, 12))
axs = axs.flatten()  # Aplatir pour un accès plus facile

# Sélectionner les dates du jour spécifique
day_dates = [dates[i] for i in day_indices]

# Couleurs pour les courbes
colors = ['blue', 'green', 'purple', 'brown']

# Tracer chaque caractéristique dans son propre sous-graphique
for feature_idx in range(4):
    ax = axs[feature_idx]
    
    # Filtrer les données pour ce jour et cette caractéristique
    true_day_values = true_values[day_indices, node_idx, feature_idx].cpu().numpy()
    pred_day_values = predictions[day_indices, node_idx, feature_idx].cpu().numpy()
    
    # Tracer les valeurs réelles et prédites
    ax.plot(day_dates, true_day_values, label="Valeurs réelles", 
            linewidth=2, color=colors[feature_idx])
    ax.plot(day_dates, pred_day_values, label="Prédictions", 
            linewidth=2, color=colors[feature_idx], linestyle='--')
    

    
    # Configurer l'axe des x pour afficher les heures
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Tick toutes les 2 heures
    
    # Ajouter une grille
    ax.grid(True, linestyle='--', alpha=0.7)
    
  
    
    # Ajouter légende et titres
    ax.legend(loc='upper right')
    ax.set_xlabel("Heure", fontsize=10)
    ax.set_ylabel(feature_names[feature_idx], fontsize=10)
    ax.set_title(f"{feature_names[feature_idx]} - Capteur {node_idx}", fontsize=12)

# Titre global
plt.suptitle(f"Prédictions vs Valeurs réelles - {specific_day.strftime('%d/%m/%Y')}", 
             fontsize=16, fontweight='bold', y=0.98)

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Laisser de l'espace pour le titre global

# Enregistrer le graphique
plt.savefig(f"prediction_sensor_{node_idx}_day_{specific_day.strftime('%Y%m%d')}.png", 
            dpi=300, bbox_inches='tight')

# Afficher le graphique
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from sklearn.metrics import r2_score
from torch_geometric_temporal.nn.recurrent import DCRNN
import matplotlib.pyplot as plt

class DCRNN_Model(pl.LightningModule):
    def __init__(self, num_nodes, num_features, num_timesteps, adjacency_matrix, edge_index, edge_weight=None, hidden_dim=128, dropout=0.3):
        super(DCRNN_Model, self).__init__()

        self.device_type = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        if edge_weight is None:
            edge_weight = torch.ones(edge_index.size(1))
        else:
            edge_weight = torch.tensor(edge_weight, dtype=torch.float32)

        self.adjacency_matrix = torch.tensor(adjacency_matrix, dtype=torch.float32, requires_grad=False).to(self.device_type)
        self.edge_index = edge_index.to(self.device_type)
        self.edge_weight = edge_weight.to(self.device_type)

        self.num_nodes = num_nodes
        self.num_features = num_features
        self.num_timesteps = num_timesteps
        self.hidden_dim = hidden_dim

        self.dcrnn = DCRNN(
            in_channels=num_features,
            out_channels=hidden_dim,
            K=2
        )

        self.output_layer = nn.Linear(hidden_dim, 4)
        nn.init.kaiming_normal_(self.output_layer.weight)

        self.dropout = nn.Dropout(dropout)

        self.train_losses = []
        self.val_losses = []
        self.train_r2 = []
        self.val_r2 = []

    def forward(self, x):
        batch_size, num_nodes, num_features, num_timesteps = x.shape
        H = torch.zeros(batch_size, num_nodes, self.hidden_dim).to(self.device_type)

        for t in range(num_timesteps):
            X_t = x[:, :, :, t]
            H_new = torch.zeros_like(H)
            for b in range(batch_size):
                H_new[b] = self.dcrnn(X_t[b], self.edge_index, self.edge_weight, H[b].detach())
            H = H_new

        H = self.dropout(H)
        output = self.output_layer(H)
        return output

    def calculate_r2(self, y_pred, y_true):
        y_pred_np = y_pred.detach().cpu().numpy().flatten()
        y_true_np = y_true.detach().cpu().numpy().flatten()
        return r2_score(y_true_np, y_pred_np)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        r2 = self.calculate_r2(y_hat, y)

        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_r2", r2, on_step=False, on_epoch=True, prog_bar=True)

        if batch_idx == 0:
            self.train_losses.append((self.current_epoch, loss.item()))
            self.train_r2.append((self.current_epoch, r2))

        return {"loss": loss, "r2": r2}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = nn.MSELoss()(y_hat, y)
        val_r2 = self.calculate_r2(y_hat, y)

        self.log("val_loss", val_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_r2", val_r2, on_step=False, on_epoch=True, prog_bar=True)

        if batch_idx == 0:
            self.val_losses.append((self.current_epoch, val_loss.item()))
            self.val_r2.append((self.current_epoch, val_r2))

        return {"val_loss": val_loss, "val_r2": val_r2}


    def on_train_epoch_end(self):
            
        train_loss = self.trainer.callback_metrics.get("train_loss")
        train_r2 = self.trainer.callback_metrics.get("train_r2")
        
        if train_loss is not None:
            # Stocker l'époque avec la valeur
            self.train_losses.append((self.current_epoch, train_loss.item()))
            print(f"Epoch {self.current_epoch}: Train Loss = {train_loss.item()}")
        
        if train_r2 is not None:
            # Stocker l'époque avec la valeur
            self.train_r2.append((self.current_epoch, train_r2.item()))
            print(f"Epoch {self.current_epoch}: Train R² = {train_r2.item()}")
    
    def on_validation_epoch_end(self):
        val_loss = self.trainer.callback_metrics.get("val_loss")
        val_r2 = self.trainer.callback_metrics.get("val_r2")
        
        if val_loss is not None:
            # Stocker l'époque avec la valeur
            self.val_losses.append((self.current_epoch, val_loss.item()))
            print(f"Epoch {self.current_epoch}: Val Loss = {val_loss.item()}")
        
        if val_r2 is not None:
            # Stocker l'époque avec la valeur
            self.val_r2.append((self.current_epoch, val_r2.item()))
            print(f"Epoch {self.current_epoch}: Val R² = {val_r2.item()}")

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-4)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
            }
        }

    def plot_losses(self):
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        if self.train_losses:
            train_epochs = [t[0] for t in self.train_losses]
            train_loss_values = [t[1] for t in self.train_losses]
            plt.plot(train_epochs, train_loss_values, 'o-', label="Train Loss")

        if self.val_losses:
            val_epochs = [t[0] for t in self.val_losses]
            val_loss_values = [t[1] for t in self.val_losses]
            plt.plot(val_epochs, val_loss_values, 'o-', label="Validation Loss")

        plt.xlabel("Epoch")
        plt.ylabel("Loss (MSE)")
        plt.title("Loss Curves")
        plt.legend()
        plt.grid()

        plt.subplot(1, 2, 2)
        if self.train_r2:
            train_r2_epochs = [t[0] for t in self.train_r2]
            train_r2_values = [t[1] for t in self.train_r2]
            plt.plot(train_r2_epochs, train_r2_values, 'o-', label="Train R²")

        if self.val_r2:
            val_r2_epochs = [t[0] for t in self.val_r2]
            val_r2_values = [t[1] for t in self.val_r2]
            plt.plot(val_r2_epochs, val_r2_values, 'o-', label="Validation R²")

        plt.xlabel("Epoch")
        plt.ylabel("R² Score")
        plt.title("R² Curves")
        plt.legend()
        plt.grid()

        plt.tight_layout()
        plt.savefig("training_curves.png")
        plt.show()



In [ ]:



# from torch_geometric_temporal.nn.recurrent import DCRNN

num_sensors = data_restructured_final.shape[1]
num_features = data_restructured_final.shape[2]
num_timesteps = 12
model1 = DCRNN_Model(num_sensors, num_features, num_timesteps, adj_matrix2, edge_index=edge_index2, dropout=0.3)



In [ ]:
# Entraînement
import pytorch_lightning as pl
trainer = pl.Trainer(max_epochs=20)
trainer.fit(model1, train_loader, val_loader, ckpt_path= None)


In [ ]:
# Test final
results = trainer.test(model1, test_loader)
print(results)

In [ ]:
model1.plot_losses()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from sklearn.metrics import r2_score

# Définir le jour spécifique à visualiser
specific_day = datetime(2021, 8, 3)  # Par exemple, le 5 mars 2025

# Créer un filtre pour sélectionner uniquement les données de ce jour
# Supposons que vous avez 12 points par jour (toutes les 2 heures)
start_date = datetime(2021, 6, 1)  # Date de début de vos données
dates = [start_date + timedelta(hours=2*i) for i in range(len(true_values))]

# Filtrer les indices correspondant au jour spécifique
day_indices = [i for i, date in enumerate(dates) if date.date() == specific_day.date()]

# Choisir un nœud spécifique
node_idx = 10

# Noms des caractéristiques
feature_names = ["Volume", "Occupation", "Niveau de congestion", "Vitesse moyenne"]

# Créer une figure 2x2 pour les 4 caractéristiques
fig, axs = plt.subplots(2, 2, figsize=(15, 12))
axs = axs.flatten()  # Aplatir pour un accès plus facile

# Sélectionner les dates du jour spécifique
day_dates = [dates[i] for i in day_indices]

# Couleurs pour les courbes
colors = ['blue', 'green', 'purple', 'brown']

# Tracer chaque caractéristique dans son propre sous-graphique
for feature_idx in range(4):
    ax = axs[feature_idx]
    
    # Filtrer les données pour ce jour et cette caractéristique
    true_day_values = true_values[day_indices, node_idx, feature_idx].cpu().numpy()
    pred_day_values = predictions[day_indices, node_idx, feature_idx].cpu().numpy()
    
    # Tracer les valeurs réelles et prédites
    ax.plot(day_dates, true_day_values, label="Valeurs réelles", 
            linewidth=2, color=colors[feature_idx])
    ax.plot(day_dates, pred_day_values, label="Prédictions", 
            linewidth=2, color=colors[feature_idx], linestyle='--')
    

    
    # Configurer l'axe des x pour afficher les heures
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))  # Tick toutes les 2 heures
    
    # Ajouter une grille
    ax.grid(True, linestyle='--', alpha=0.7)
    
  
    
    # Ajouter légende et titres
    ax.legend(loc='upper right')
    ax.set_xlabel("Heure", fontsize=10)
    ax.set_ylabel(feature_names[feature_idx], fontsize=10)
    ax.set_title(f"{feature_names[feature_idx]} - Capteur {node_idx}", fontsize=12)

# Titre global
plt.suptitle(f"Prédictions vs Valeurs réelles - {specific_day.strftime('%d/%m/%Y')}", 
             fontsize=16, fontweight='bold', y=0.98)

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Laisser de l'espace pour le titre global

# Enregistrer le graphique
plt.savefig(f"prediction_sensor_{node_idx}_day_{specific_day.strftime('%Y%m%d')}.png", 
            dpi=300, bbox_inches='tight')

# Afficher le graphique
plt.show()

In [ ]:
# Passer l'ensemble de test dans le modèle
model1.eval()  # Mettre le modèle en mode évaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)
# Test loop
with torch.no_grad():
    predictions = []
    true_values = []
    for batch in test_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)  # Déplacer les données sur le même appareil que le modèle
        y_hat = model1(x)
        predictions.append(y_hat)
        true_values.append(y)

# Concaténer les résultats
predictions = torch.cat(predictions, dim=0)
true_values = torch.cat(true_values, dim=0)

# Afficher les résultats
print("Predictions:", predictions)
print("True values:", true_values)

In [ ]:
import matplotlib.pyplot as plt

# Afficher les prédictions et les vraies valeurs pour un nœud donné
node_idx = 50 # Choisir un nœud spécifique
plt.figure(figsize=(12, 6))  # Taille de la figure
plt.plot(true_values[:, node_idx, 0].cpu().numpy(), label="True values")
plt.plot(predictions[:, node_idx, 0].cpu().numpy(), label="Predictions")

# Ajuster l'échelle de l'axe des temps
num_time_steps = len(true_values[:, node_idx, 0])  # Nombre total de time steps
step = 200  # Afficher une valeur tous les 100 time steps
plt.xticks(range(0, num_time_steps, step), rotation=45)  # Rotation pour une meilleure lisibilité

# Ajouter une grille
plt.grid(True, linestyle='--', alpha=0.6)

plt.legend()
plt.xlabel("Time steps")
plt.ylabel("Value")
plt.title("Predictions vs True values")
plt.show()

In [ ]:
pip install --upgrade optuna pytorch-lightning optuna-integration

In [ ]:
model2.plot_losses()

In [ ]:
# Test final
results = trainer.test(model2, test_loader)
print(results)

In [ ]:
pip install optuna 

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import joblib
import torch
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

class OptimizedASTGCNModel(pl.LightningModule):
    def __init__(self, num_nodes, edge_index, num_features=11, num_timesteps=12, 
                 output_size=4, trial=None):
        super().__init__()
        self.save_hyperparameters()
        
        # Paramètres optimisés
        self.params = self.get_optimized_params(trial)
        
        self.astgcn = ASTGCN(
            nb_block=self.params['nb_block'],
            in_channels=num_features,
            K=self.params['K'],
            nb_chev_filter=self.params['nb_chev_filter'],
            nb_time_filter=self.params['nb_time_filter'],
            time_strides=1,
            num_for_predict=num_timesteps,
            len_input=num_timesteps,
            num_of_vertices=num_nodes,
        )
        
        # Architecture dynamique
        self.adapter = self.create_adapter_layer()
        self.output_layer = nn.Linear(self.params['adapter_output_dim'], num_nodes * output_size)
        self.dropout = nn.Dropout(self.params['dropout'])
        
        # Tracking des métriques
        self.metrics = {
            'train': {'loss': [], 'r2': []},
            'val': {'loss': [], 'r2': []},
            'test': {'loss': None, 'r2': None}
        }

    def get_optimized_params(self, trial):
        if trial:
            return {
                'nb_block': trial.suggest_int('nb_block', 1, 3),
                'K': trial.suggest_int('K', 2, 5),
                'nb_chev_filter': trial.suggest_categorical('nb_chev_filter', [32, 64, 128]),
                'nb_time_filter': trial.suggest_categorical('nb_time_filter', [32, 64, 128]),
                'dropout': trial.suggest_float('dropout', 0.1, 0.5),
                'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
                'weight_decay': trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True),
                'use_layer_norm': trial.suggest_categorical('use_layer_norm', [True, False]),
                'adapter_hidden_dim': trial.suggest_categorical('adapter_hidden_dim', [256, 512, 1024]),
                'adapter_output_dim': None  # Será calculé dans create_adapter_layer
            }
        else:
            # Valeurs par défaut
            return {
                'nb_block': 2,
                'K': 3,
                'nb_chev_filter': 64,
                'nb_time_filter': 64,
                'dropout': 0.2,
                'learning_rate': 1e-3,
                'weight_decay': 1e-5,
                'use_layer_norm': True,
                'adapter_hidden_dim': 512,
                'adapter_output_dim': 512
            }

    def create_adapter_layer(self):
        layers = []
        input_dim = self.hparams.num_nodes * self.params['nb_time_filter']
        
        if self.params['adapter_hidden_dim'] > 0:
            layers.append(nn.Linear(input_dim, self.params['adapter_hidden_dim']))
            if self.params['use_layer_norm']:
                layers.append(nn.LayerNorm(self.params['adapter_hidden_dim']))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(self.params['dropout']/2))
            self.params['adapter_output_dim'] = self.params['adapter_hidden_dim']
        else:
            self.params['adapter_output_dim'] = input_dim
        
        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.to(self.device)
        self.hparams.edge_index = self.hparams.edge_index.to(self.device)
        
        x = self.astgcn(x, self.hparams.edge_index)
        x = x.reshape(x.size(0), -1)
        x = self.adapter(x)
        x = self.output_layer(x)
        x = self.dropout(x)
        return x.reshape(x.size(0), self.hparams.num_nodes, self.hparams.output_size)

    def configure_optimizers(self):
        optimizer = optim.AdamW(
            self.parameters(),
            lr=self.params['learning_rate'],
            weight_decay=self.params['weight_decay']
        )
        
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=5,
            verbose=True
        )
        
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss',
                'interval': 'epoch',
                'frequency': 1
            }
        }

    def log_metrics(self, phase, loss, y, y_hat):
        self.log(f"{phase}_loss", loss, prog_bar=True)
        r2 = r2_score(y.cpu().numpy().flatten(), y_hat.cpu().detach().numpy().flatten())
        self.log(f"{phase}_r2", r2, prog_bar=True)
        
        if phase in ['train', 'val']:
            self.metrics[phase]['loss'].append(loss.item())
            self.metrics[phase]['r2'].append(r2)
        else:
            self.metrics[phase]['loss'] = loss.item()
            self.metrics[phase]['r2'] = r2

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log_metrics('train', loss, y, y_hat)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log_metrics('val', loss, y, y_hat)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log_metrics('test', loss, y, y_hat)
        return loss

    def plot_metrics(self):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss plot
        ax1.plot(self.metrics['train']['loss'], label='Train Loss')
        ax1.plot(self.metrics['val']['loss'], label='Validation Loss')
        ax1.set_title('Loss Evolution')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('MSE Loss')
        ax1.legend()
        ax1.grid()
        
        # R2 plot
        ax2.plot(self.metrics['train']['r2'], label='Train R²')
        ax2.plot(self.metrics['val']['r2'], label='Validation R²')
        ax2.set_title('R² Score Evolution')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('R² Score')
        ax2.legend()
        ax2.grid()
        
        plt.tight_layout()
        plt.savefig('training_metrics.png')
        plt.show()

def optimize_hyperparameters():
    study = optuna.create_study(
        direction='minimize',
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner()
    )
    
    def objective(trial):
        model = OptimizedASTGCNModel(
            num_nodes=num_sensors,
            edge_index=edge_index1,
            num_features=num_features,
            num_timesteps=num_timesteps,
            output_size=4,
            trial=trial
        )
        
        trainer = pl.Trainer(
            max_epochs=30,
            accelerator='auto',
            devices=1 if torch.cuda.is_available() else None,
            callbacks=[
                PyTorchLightningPruningCallback(trial, monitor='val_loss'),
                pl.callbacks.EarlyStopping(monitor='val_loss', patience=7)
            ],
            enable_progress_bar=True,
            enable_model_summary=True
        )
        
        trainer.fit(model)
        return trainer.callback_metrics['val_loss'].item()
    
    study.optimize(objective, n_trials=50, timeout=3600*3)
    
    # Sauvegarde et visualisation
    joblib.dump(study, 'astgcn_study.pkl')
    plot_optimization_history(study).show()
    plot_param_importances(study).show()
    
    return study.best_params

In [ ]:
def train_final_model(best_params):
    model = OptimizedASTGCNModel(
        num_nodes=num_sensors,
        edge_index=edge_index1,
        num_features=num_features,
        num_timesteps=num_timesteps,
        output_size=4,
        trial=None
    )
    
    # Mise à jour des paramètres avec les meilleures valeurs
    model.hparams.params.update(best_params)
    
    trainer = pl.Trainer(
        max_epochs=20,
        accelerator='auto',
        devices=1 if torch.cuda.is_available() else None,
        callbacks=[
            pl.callbacks.ModelCheckpoint(monitor='val_loss'),
            pl.callbacks.LearningRateMonitor()
        ]
    )
    
    trainer.fit(model)
    test_results = trainer.test(model)
    
    # Sauvegarde du modèle
    torch.save(model.state_dict(), 'best_astgcn_model.pth')
    
    # Visualisation
    model.plot_metrics()
    
    return model, test_results


In [ ]:
import logging
# Exécution complète

if __name__ == '__main__':
    # Phase d'optimisation
    best_params = optimize_hyperparameters()
    print("Meilleurs paramètres trouvés :", best_params)
    
    # Phase d'entraînement final
    final_model, test_results = train_final_model(best_params)
    print("Résultats du test :", test_results)

In [ ]:
pip install optuna-integration[pytorch_lightning]

In [ ]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import plot_optimization_history, plot_param_importances
import joblib
import torch
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import logging

# Configuration du logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class OptimizedASTGCNModel(pl.LightningModule):
    def __init__(self, num_nodes, edge_index, num_features=11, num_timesteps=12, 
                 output_size=4, trial=None):
        super().__init__()
        self.save_hyperparameters()
        self.edge_index = edge_index
        
        self.params = self.get_optimized_params(trial)
        logger.info(f"Initialisation du modèle avec num_nodes={num_nodes}")
        logger.debug(f"Paramètres optimisés: {self.params}")

        try:
            self.astgcn = ASTGCN(
                nb_block=self.params['nb_block'],
                in_channels=num_features,
                K=self.params['K'],
                nb_chev_filter=self.params['nb_chev_filter'],
                nb_time_filter=self.params['nb_time_filter'],
                time_strides=1,
                num_for_predict=num_timesteps,
                len_input=num_timesteps,
                num_of_vertices=num_nodes,
            )

            self.adapter = self.create_adapter_layer()
            adapter_output_dim = self.params['adapter_output_dim']
            self.output_layer = nn.Linear(adapter_output_dim, num_nodes * output_size)
            self.dropout = nn.Dropout(self.params['dropout'])
        
        except Exception as e:
            logger.error(f"Erreur lors de l'initialisation du modèle: {str(e)}")
            raise

    def create_adapter_layer(self):
        layers = []
        input_dim = self.hparams.num_nodes * self.params['nb_time_filter']

        if input_dim <= 0:
            raise ValueError(f"Dimension d'entrée invalide: {input_dim}")
        
        if self.params['adapter_hidden_dim'] > 0:
            layers.append(nn.Linear(input_dim, self.params['adapter_hidden_dim']))
            if self.params['use_layer_norm']:
                layers.append(nn.LayerNorm(self.params['adapter_hidden_dim']))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(self.params['dropout'] / 2))
            self.params['adapter_output_dim'] = self.params['adapter_hidden_dim']
        else:
            self.params['adapter_output_dim'] = input_dim
        
        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.to(self.device)
        self.edge_index = self.edge_index.to(self.device)
        
        x = self.astgcn(x, self.edge_index)
        x = x.reshape(x.size(0), -1)
        x = self.adapter(x)
        x = self.output_layer(x)
        x = self.dropout(x)
        return x.reshape(x.size(0), self.hparams.num_nodes, self.hparams.output_size)

    def configure_optimizers(self):
        optimizer = optim.AdamW(
            self.parameters(),
            lr=self.params['learning_rate'],
            weight_decay=self.params['weight_decay']
        )
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5, verbose=True
        )
        return {'optimizer': optimizer, 'lr_scheduler': {'scheduler': scheduler, 'monitor': 'val_loss'}}

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log("test_loss", loss, prog_bar=True)
        return loss

def optimize_hyperparameters():
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
    
    def objective(trial):
        model = OptimizedASTGCNModel(
            num_nodes=num_sensors,
            edge_index=edge_index1,
            num_features=num_features,
            num_timesteps=num_timesteps,
            output_size=4,
            trial=trial
        )
        
        trainer = pl.Trainer(
            max_epochs=30,
            accelerator='auto',
            devices=1 if torch.cuda.is_available() else None,
            callbacks=[
                PyTorchLightningPruningCallback(trial, monitor='val_loss'),
                pl.callbacks.EarlyStopping(monitor='val_loss', patience=7)
            ],
            enable_progress_bar=False
        )
        trainer.fit(model)
        return trainer.callback_metrics['val_loss'].item()
    
    study.optimize(objective, n_trials=50, timeout=10800)
    joblib.dump(study, 'astgcn_study.pkl')
    plot_optimization_history(study).show()
    plot_param_importances(study).show()
    
    return study.best_params

def train_final_model(best_params):
    model = OptimizedASTGCNModel(
        num_nodes=num_sensors,
        edge_index=edge_index1,
        num_features=num_features,
        num_timesteps=num_timesteps,
        output_size=4
    )
    model.hparams.params.update(best_params)
    
    trainer = pl.Trainer(
        max_epochs=100,
        accelerator='auto',
        devices=1 if torch.cuda.is_available() else None,
        callbacks=[
            pl.callbacks.ModelCheckpoint(monitor='val_loss'),
            pl.callbacks.LearningRateMonitor()
        ]
    )
    trainer.fit(model)
    trainer.test(model)
    torch.save(model.state_dict(), 'best_astgcn_model.pth')
    
    return model

if __name__ == '__main__':
    best_params = optimize_hyperparameters()
    print("Meilleurs paramètres trouvés :", best_params)
    final_model = train_final_model(best_params)


In [ ]:
pip install optuna-integration[pytorch_lightning]

In [ ]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping
import pytorch_lightning as pl

def objective(trial):
    # Hyperparamètres à optimiser
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    nb_chev_filter = trial.suggest_categorical('nb_chev_filter', [16, 32, 64])
    nb_time_filter = trial.suggest_categorical('nb_time_filter', [16, 32, 64])
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Créer le modèle avec les hyperparamètres suggérés
    model = ASTGCNModel(
        num_nodes=421,
        edge_index=edge_index1,
        num_features=11,
        num_timesteps=24,
        output_features=4,
        dropout=dropout,
        nb_chev_filter=nb_chev_filter,
        nb_time_filter=nb_time_filter
    )
    
    # Mettre à jour les paramètres de l'optimiseur
    model.configure_optimizers = lambda: optim.Adam(
        model.parameters(), 
        lr=lr, 
        weight_decay=weight_decay
    )

    # Configuration des callbacks
    early_stopping = EarlyStopping(
        monitor="val_loss",
        patience=5,
        mode="min",
        verbose=True
    )
    

    # Configuration de l'entraîneur
    trainer = pl.Trainer(
        max_epochs=30,
        callbacks=[early_stopping],
        enable_progress_bar=True,
        enable_model_summary=True,
        accelerator='auto',
        devices='auto'
    )

    # Entraîner le modèle
    trainer.fit(model)

    # Retourner la meilleure valeur de val_loss
    return trainer.callback_metrics["val_loss"].item()

# Étude Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, timeout=3600)

# Afficher les résultats
print("\nRésultats de l'optimisation:")
print("Nombre d'essais: ", len(study.trials))
print("Meilleur essai:")
trial = study.best_trial
print(f"  Valeur (val_loss): {trial.value:.4f}")
print("  Paramètres: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Entraîner le modèle final avec les meilleurs hyperparamètres
best_params = study.best_params
final_model = ASTGCNModel(
    num_nodes=421,
    edge_index=edge_index1,
    num_features=11,
    num_timesteps=24,
    output_features=4,
    dropout=best_params['dropout'],
    nb_chev_filter=best_params['nb_chev_filter'],
    nb_time_filter=best_params['nb_time_filter']
)

# Configuration finale
final_trainer = pl.Trainer(
    max_epochs=100,
    accelerator='auto',
    devices=1,
    enable_progress_bar=True,
    enable_model_summary=True
)

# Entraînement final
final_trainer.fit(final_model)

# Visualisation des résultats
final_model.plot_losses()

# Test final
test_results = final_trainer.test(final_model)
print("\nRésultats du test:")
print(f"Test Loss: {test_results[0]['test_loss']:.4f}")

In [ ]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.environ['PYTHONHASHSEED'] = '42'

import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from torch_geometric_temporal.nn import MTGNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, List, Tuple

# Seed everything pour reproductibilité
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False)

seed_everything()

# Paramètres
NUM_SENSORS = 421
NUM_FEATURES = 11
NUM_TIMESTEPS = 12
HORIZON = 1
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 20

# -------------------------------------------------
# 1. Préparation des données (simulées pour l'exemple)
# -------------------------------------------------
# Génération de données aléatoires pour l'exemple
data_tensor = torch.tensor(data_restructured_final, dtype=torch.float32)

def create_sequences(data, seq_len, horizon):
    sequences = []
    targets = []
    for i in range(len(data) - seq_len - horizon):
        seq = data[i:i+seq_len]  # (seq_len, num_nodes, num_features)
        target = data[i+seq_len, :, :4]  # (num_nodes, 4)
        sequences.append(seq)
        targets.append(target)
    return torch.stack(sequences), torch.stack(targets)

sequences, targets = create_sequences(data_tensor, NUM_TIMESTEPS, HORIZON)

# Split des données
train_seq, temp_seq, train_targ, temp_targ = train_test_split(
    sequences, targets, test_size=0.2, random_state=42
)
val_seq, test_seq, val_targ, test_targ = train_test_split(
    temp_seq, temp_targ, test_size=0.5, random_state=42
)

# -------------------------------------------------
# 2. Dataset et DataLoader
# -------------------------------------------------
class TrafficDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

# Création des datasets
train_dataset = TrafficDataset(train_seq, train_targ)
val_dataset = TrafficDataset(val_seq, val_targ)
test_dataset = TrafficDataset(test_seq, test_targ)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=4, persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=4, persistent_workers=True)

In [ ]:
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import torch
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import torch.nn.functional as F

class TrafficModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.variable_names = ['volume', 'occupation', 'congestion_level', 'speed_avg']
        
        self.feature_transform = nn.Sequential(
            nn.Conv2d(
                in_channels=NUM_FEATURES, 
                out_channels=32,
                kernel_size=1
            ),
            nn.ReLU()
        )
        
        self.model = MTGNN(
            gcn_true=True,
            build_adj=True,
            gcn_depth=2,
            num_nodes=NUM_SENSORS,
            kernel_size=2,
            kernel_set=[1, 2],
            subgraph_size=20,
            node_dim=40,
            dilation_exponential=1,
            conv_channels=32,
            residual_channels=32,
            skip_channels=64,
            end_channels=128,
            seq_length=NUM_TIMESTEPS,
            in_dim=32,
            out_dim=4,
            layers=2,
            propalpha=0.05,
            tanhalpha=3,
            layer_norm_affline=True,
            dropout=0.2
        )
        
        self.loss = nn.MSELoss()
        self.save_hyperparameters()
        
        # Métriques globales
        self.train_metrics = {'epoch': [], 'loss': [], 'mse': [], 'rmse': [], 'mae': [], 'r2': []}
        self.val_metrics = {'epoch': [], 'loss': [], 'mse': [], 'rmse': [], 'mae': [], 'r2': []}
        self.test_metrics = {'epoch': [], 'loss': [], 'mse': [], 'rmse': [], 'mae': [], 'r2': []}
        
        # Métriques par variable
        self.train_var_metrics = {var: {'mse': [], 'rmse': [], 'mae': [], 'r2': []} for var in self.variable_names}
        self.val_var_metrics = {var: {'mse': [], 'rmse': [], 'mae': [], 'r2': []} for var in self.variable_names}
        self.test_var_metrics = {var: {'mse': [], 'rmse': [], 'mae': [], 'r2': []} for var in self.variable_names}

    def forward(self, x):
        x = x.permute(0, 3, 2, 1)
        x = self.feature_transform(x)
        output = self.model(x)
        return output.squeeze(-1).permute(0, 2, 1)

    def _compute_metrics(self, y_hat, y):
        y_hat_np = y_hat.detach().cpu().numpy().flatten()
        y_np = y.detach().cpu().numpy().flatten()
        
        return {
            'mse': mean_squared_error(y_np, y_hat_np),
            'rmse': np.sqrt(mean_squared_error(y_np, y_hat_np)),
            'mae': mean_absolute_error(y_np, y_hat_np),
            'r2': r2_score(y_np, y_hat_np)
        }

    def _compute_metrics_by_variable(self, y_hat, y):
        metrics = {}
        for i, var_name in enumerate(self.variable_names):
            y_hat_var = y_hat[..., i].flatten().detach().cpu().numpy()
            y_var = y[..., i].flatten().detach().cpu().numpy()
            
            metrics[var_name] = {
                'mse': mean_squared_error(y_var, y_hat_var),
                'rmse': np.sqrt(mean_squared_error(y_var, y_hat_var)),
                'mae': mean_absolute_error(y_var, y_hat_var),
                'r2': r2_score(y_var, y_hat_var)
            }
        return metrics

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        
        # Calcul des métriques
        metrics = self._compute_metrics(y_hat, y)
        var_metrics = self._compute_metrics_by_variable(y_hat, y)
        
        # Logging global
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_mse', metrics['mse'], prog_bar=False)
        self.log('train_rmse', metrics['rmse'], prog_bar=True)
        self.log('train_mae', metrics['mae'], prog_bar=False)
        self.log('train_r2', metrics['r2'], prog_bar=True)
        
        # Logging par variable
        for var_name in self.variable_names:
            self.log(f'train_{var_name}_mse', var_metrics[var_name]['mse'], prog_bar=False)
            self.log(f'train_{var_name}_rmse', var_metrics[var_name]['rmse'], prog_bar=False)
            self.log(f'train_{var_name}_mae', var_metrics[var_name]['mae'], prog_bar=False)
            self.log(f'train_{var_name}_r2', var_metrics[var_name]['r2'], prog_bar=False)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        
        # Calcul des métriques
        metrics = self._compute_metrics(y_hat, y)
        var_metrics = self._compute_metrics_by_variable(y_hat, y)
        
        # Logging global
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_mse', metrics['mse'], prog_bar=False)
        self.log('val_rmse', metrics['rmse'], prog_bar=True)
        self.log('val_mae', metrics['mae'], prog_bar=False)
        self.log('val_r2', metrics['r2'], prog_bar=True)
        
        # Logging par variable
        for var_name in self.variable_names:
            self.log(f'val_{var_name}_mse', var_metrics[var_name]['mse'], prog_bar=False)
            self.log(f'val_{var_name}_rmse', var_metrics[var_name]['rmse'], prog_bar=False)
            self.log(f'val_{var_name}_mae', var_metrics[var_name]['mae'], prog_bar=False)
            self.log(f'val_{var_name}_r2', var_metrics[var_name]['r2'], prog_bar=False)
        
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        
        # Calcul des métriques
        metrics = self._compute_metrics(y_hat, y)
        var_metrics = self._compute_metrics_by_variable(y_hat, y)
        
        # Logging global
        self.log('test_loss', loss)
        self.log('test_mse', metrics['mse'])
        self.log('test_rmse', metrics['rmse'])
        self.log('test_mae', metrics['mae'])
        self.log('test_r2', metrics['r2'])
        
        # Logging par variable
        for var_name in self.variable_names:
            self.log(f'test_{var_name}_mse', var_metrics[var_name]['mse'])
            self.log(f'test_{var_name}_rmse', var_metrics[var_name]['rmse'])
            self.log(f'test_{var_name}_mae', var_metrics[var_name]['mae'])
            self.log(f'test_{var_name}_r2', var_metrics[var_name]['r2'])
        
        return {
            'test_loss': loss,
            'metrics': metrics,
            'var_metrics': var_metrics
        }

    def on_train_epoch_end(self):
        metrics = self.trainer.callback_metrics
        epoch = self.current_epoch
        
        # Stockage métriques globales
        self.train_metrics['epoch'].append(epoch)
        for key in ['loss', 'mse', 'rmse', 'mae', 'r2']:
            self.train_metrics[key].append(metrics[f'train_{key}'].item())
        
        # Stockage métriques par variable
        for var_name in self.variable_names:
            for metric in ['mse', 'rmse', 'mae', 'r2']:
                self.train_var_metrics[var_name][metric].append(metrics[f'train_{var_name}_{metric}'].item())
        
        # Affichage
        print(f"\nEpoch {epoch} - Train Metrics:")
        print(f"Global - Loss: {metrics['train_loss'].item():.4f} | MSE: {metrics['train_mse'].item():.4f} | RMSE: {metrics['train_rmse'].item():.4f} | MAE: {metrics['train_mae'].item():.4f} | R²: {metrics['train_r2'].item():.4f}")
        
        for var_name in self.variable_names:
            print(f"{var_name}:")
            print(f"    MSE: {metrics[f'train_{var_name}_mse'].item():.4f}, RMSE: {metrics[f'train_{var_name}_rmse'].item():.4f}")
            print(f"    MAE: {metrics[f'train_{var_name}_mae'].item():.4f}, R²: {metrics[f'train_{var_name}_r2'].item():.4f}")
            print("---")

    def on_validation_epoch_end(self):
        metrics = self.trainer.callback_metrics
        epoch = self.current_epoch
        
        # Stockage métriques globales
        self.val_metrics['epoch'].append(epoch)
        for key in ['loss', 'mse', 'rmse', 'mae', 'r2']:
            self.val_metrics[key].append(metrics[f'val_{key}'].item())
        
        # Stockage métriques par variable
        for var_name in self.variable_names:
            for metric in ['mse', 'rmse', 'mae', 'r2']:
                self.val_var_metrics[var_name][metric].append(metrics[f'val_{var_name}_{metric}'].item())
        
        # Affichage formaté comme dans la capture d'écran
        print(f"\nEpoch {epoch} - Validation Metrics by Variable:")
        for var_name in self.variable_names:
            print(f"{var_name}:")
            print(f"    MSE: {metrics[f'val_{var_name}_mse'].item():.4f}, RMSE: {metrics[f'val_{var_name}_rmse'].item():.4f}")
            print(f"    MAE: {metrics[f'val_{var_name}_mae'].item():.4f}, R²: {metrics[f'val_{var_name}_r2'].item():.4f}")
            print("---")

    def on_test_epoch_end(self):
        metrics = self.trainer.callback_metrics
        
        # Stockage métriques globales
        self.test_metrics['epoch'].append(0)
        for key in ['loss', 'mse', 'rmse', 'mae', 'r2']:
            self.test_metrics[key].append(metrics[f'test_{key}'].item())
        
        # Stockage métriques par variable
        for var_name in self.variable_names:
            for metric in ['mse', 'rmse', 'mae', 'r2']:
                self.test_var_metrics[var_name][metric].append(metrics[f'test_{var_name}_{metric}'].item())
        
        # Affichage final comme dans la capture d'écran
        print("\nFinal Test Metrics by Variable:")
        for var_name in self.variable_names:
            print(f"{var_name}:")
            print(f"    MSE: {metrics[f'test_{var_name}_mse'].item():.4f}, RMSE: {metrics[f'test_{var_name}_rmse'].item():.4f}")
            print(f"    MAE: {metrics[f'test_{var_name}_mae'].item():.4f}, R²: {metrics[f'test_{var_name}_r2'].item():.4f}")
            print("---")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=LEARNING_RATE)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, 
            mode='min',
            factor=0.5,
            patience=3,
            verbose=True
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
            }
        }

    def get_adjacency_matrix(self):
        """Retourne la matrice d'adjacence apprise par le modèle"""
        graph_constructor = self.model._graph_constructor
        
        if hasattr(graph_constructor, 'adj'):
            return graph_constructor.adj.detach().cpu().numpy()
        
        if hasattr(graph_constructor, 'build_adjacency'):
            with torch.no_grad():
                return graph_constructor.build_adjacency().detach().cpu().numpy()
        
        if hasattr(graph_constructor, '_embedding1') and hasattr(graph_constructor, '_embedding2'):
            with torch.no_grad():
                nodevec1 = graph_constructor._embedding1.weight
                nodevec2 = graph_constructor._embedding2.weight
                adj = torch.mm(nodevec1, nodevec2.transpose(1, 0))
                adj = F.softmax(F.relu(adj), dim=1)
                return adj.detach().cpu().numpy()
        
        print("Structure du graph constructor:")
        for name, param in graph_constructor.named_parameters():
            print(f"{name}: {param.shape}")
        
        raise AttributeError("Impossible de trouver la matrice d'adjacence dans le graph constructor")

    def plot_weighted_graph(self, top_k=20, figsize=(15, 12)):
        adj_matrix = self.get_adjacency_matrix()
        sorted_indices = np.dstack(np.unravel_index(np.argsort(-adj_matrix.ravel()), adj_matrix.shape))[0]
        
        G = nx.Graph()
        for i in range(min(top_k, len(sorted_indices))):
            u, v = sorted_indices[i]
            weight = adj_matrix[u, v]
            if weight > 0:
                G.add_edge(u, v, weight=weight)
        
        pos = nx.spring_layout(G, k=0.5)
        plt.figure(figsize=figsize)
        
        nx.draw_networkx_nodes(G, pos, node_size=800, node_color='lightcoral', alpha=0.9)
        
        edges = G.edges(data=True)
        widths = [2 + 5 * d['weight'] for _, _, d in edges]
        nx.draw_networkx_edges(G, pos, width=widths, alpha=0.6, edge_color='dimgray')
        
        nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
        
        edge_labels = {(u, v): f"{d['weight']:.2f}" for u, v, d in edges}
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
        
        plt.title(f"Top {top_k} connexions les plus fortes")
        plt.axis('off')
        plt.tight_layout()
        plt.savefig('weighted_graph.png', dpi=300)
        plt.show()
    
    def plot_metrics(self):
        """Trace les courbes d'apprentissage globales"""
        try:
            plt.figure(figsize=(18, 12))
            
            # Loss
            plt.subplot(2, 2, 1)
            plt.plot(self.train_metrics['epoch'], self.train_metrics['loss'], 'b-', label='Train')
            plt.plot(self.val_metrics['epoch'], self.val_metrics['loss'], 'r-', label='Validation')
            plt.title('Loss')
            plt.xlabel('Epoch')
            plt.grid(True)
            plt.legend()
            
            # MSE
            plt.subplot(2, 2, 2)
            plt.plot(self.train_metrics['epoch'], self.train_metrics['mse'], 'b-', label='Train')
            plt.plot(self.val_metrics['epoch'], self.val_metrics['mse'], 'r-', label='Validation')
            plt.title('MSE')
            plt.xlabel('Epoch')
            plt.grid(True)
            plt.legend()
            
            # RMSE
            plt.subplot(2, 2, 3)
            plt.plot(self.train_metrics['epoch'], self.train_metrics['rmse'], 'b-', label='Train')
            plt.plot(self.val_metrics['epoch'], self.val_metrics['rmse'], 'r-', label='Validation')
            plt.title('RMSE')
            plt.xlabel('Epoch')
            plt.grid(True)
            plt.legend()
            
            # R²
            plt.subplot(2, 2, 4)
            plt.plot(self.train_metrics['epoch'], self.train_metrics['r2'], 'b-', label='Train')
            plt.plot(self.val_metrics['epoch'], self.val_metrics['r2'], 'r-', label='Validation')
            plt.title('R² Score')
            plt.xlabel('Epoch')
            plt.grid(True)
            plt.legend()
            
            plt.tight_layout()
            plt.savefig('final_metrics_plot.png', dpi=300)
            plt.show()
            
        except Exception as e:
            print(f"Erreur lors du tracé: {str(e)}")

    def plot_variable_metrics(self):
        """Trace les métriques pour chaque variable séparément"""
        fig, axs = plt.subplots(4, 4, figsize=(20, 15))
        
        for i, var_name in enumerate(self.variable_names):
            # MSE
            axs[0, i].plot(self.train_var_metrics[var_name]['mse'], 'b-', label="Train")
            axs[0, i].plot(self.val_var_metrics[var_name]['mse'], 'r-', label="Val")
            axs[0, i].set_title(f"{var_name} - MSE")
            axs[0, i].grid()
            
            # RMSE
            axs[1, i].plot(self.train_var_metrics[var_name]['rmse'], 'b-', label="Train")
            axs[1, i].plot(self.val_var_metrics[var_name]['rmse'], 'r-', label="Val")
            axs[1, i].set_title(f"{var_name} - RMSE")
            axs[1, i].grid()
            
            # MAE
            axs[2, i].plot(self.train_var_metrics[var_name]['mae'], 'b-', label="Train")
            axs[2, i].plot(self.val_var_metrics[var_name]['mae'], 'r-', label="Val")
            axs[2, i].set_title(f"{var_name} - MAE")
            axs[2, i].grid()
            
            # R2
            axs[3, i].plot(self.train_var_metrics[var_name]['r2'], 'b-', label="Train")
            axs[3, i].plot(self.val_var_metrics[var_name]['r2'], 'r-', label="Val")
            axs[3, i].set_title(f"{var_name} - R²")
            axs[3, i].grid()
        
        plt.tight_layout()
        plt.savefig("variable_metrics.png")
        plt.show()



In [ ]:
# -------------------------------------------------
# 4. Entraînement et évaluation
# -------------------------------------------------
model = TrafficModel()

trainer = pl.Trainer(
    max_epochs=EPOCHS,
    accelerator="auto",
    devices="auto",
    enable_progress_bar=True,
    deterministic=False,
    callbacks=[
        pl.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            mode="min"
        ),
        pl.callbacks.ModelCheckpoint(
            monitor="val_loss",
            filename="best-checkpoint",
            save_top_k=1,
            mode="min"
        ),
        pl.callbacks.LearningRateMonitor()
    ],
    log_every_n_steps=10
)

# Entraînement
try:
    print("Début de l'entraînement...")
    trainer.fit(model, train_loader, val_loader)
    
    # Test du meilleur modèle
    print("\nÉvaluation sur le test set...")
    best_model = TrafficModel.load_from_checkpoint(
        trainer.checkpoint_callback.best_model_path
    )
    
    
    # Sauvegarde des métriques dans un DataFrame
    train_df = pd.DataFrame(best_model.train_metrics)
    val_df = pd.DataFrame(best_model.val_metrics)
    test_df = pd.DataFrame(best_model.test_metrics)
    
    # Affichage des métriques finales
    print("\nRésumé des métriques finales:")
    print("\nEntraînement:")
    print(train_df.tail())
    print("\nValidation:")
    print(val_df.tail())
    print("\nTest:")
    print(test_df)
    
except Exception as e:
    print(f"\nErreur pendant l'entraînement : {str(e)}")
    raise e

In [ ]:
# -- 9. Visualisation ( étendue à toutes les cibles) --
print("Generating visualizations for all target variables...")

# 1. Courbes d'apprentissage (Loss)
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
best_epoch_marker = len(val_losses) - epochs_no_improve - 1 if best_model_state and epochs_no_improve < len(val_losses) else len(val_losses) - 1
plt.axvline(x=best_epoch_marker, color='r', linestyle='--', label=f'Best Epoch ({best_epoch_marker+1})')
plt.title('Model2 Training and Validation Loss (GAT Version)')  # Mise à jour du titre
plt.xlabel('Epochs')
plt.ylabel('Huber Loss')
plt.legend()
plt.grid(True)
plt.savefig('training_validation_loss_gat.png')
print("Saved: training_validation_loss_gat.png")
plt.close()

In [ ]:
A = model.plot_weighted_graph(top_k=421)

In [ ]:
print("Epochs train:", len(model.train_metrics['epoch']))
print("Loss train:", len(model.train_metrics['loss']))
print("Epochs val:", len(model.val_metrics['epoch']))
print("Loss val:", len(model.val_metrics['loss']))

In [ ]:
# Test final
results = trainer.test(model, test_loader)
print(results)

In [ ]:
model.plot_metrics()

In [ ]:
# Passer l'ensemble de test dans le modèle
model.eval()  # Mettre le modèle en mode évaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Test loop
with torch.no_grad():
    predictions = []
    true_values = []
    for batch in test_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)  # Déplacer les données sur le même appareil que le modèle
        y_hat = model(x)
        predictions.append(y_hat)
        true_values.append(y)

# Concaténer les résultats
predictions = torch.cat(predictions, dim=0)
true_values = torch.cat(true_values, dim=0)

# Afficher les résultats
print("Predictions:", predictions)
print("True values:", true_values)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Vérification des shapes
print(f"Shape de true_values: {true_values.shape}")
print(f"Shape de predictions: {predictions.shape}")

# Conversion des tenseurs si nécessaire
if hasattr(true_values, 'cpu'):  # Si c'est un tenseur PyTorch
    true_values = true_values.cpu().numpy()
if hasattr(predictions, 'cpu'):
    predictions = predictions.cpu().numpy()

# Redimensionnement des prédictions si nécessaire (supprimer la dimension inutile)
if predictions.ndim == 4 and predictions.shape[2] == 1:  # Si shape [1170, 421, 1, 4]
    predictions = predictions.squeeze(2)  # Devient [1170, 421, 4]

# Définir le jour spécifique à visualiser
specific_day = datetime(2021, 9, 2)  # Adaptez cette date

# Créer les dates
start_date = datetime(2021, 6, 1)  # Date de début des données
dates = [start_date + timedelta(hours=2*i) for i in range(true_values.shape[0])]

# Filtrer les indices du jour spécifique
day_indices = [i for i, date in enumerate(dates) if date.date() == specific_day.date()]

# Vérifier qu'on a des données pour ce jour
if not day_indices:
    raise ValueError(f"Aucune donnée disponible pour le {specific_day.strftime('%d/%m/%Y')}")

# Choisir un nœud spécifique
node_idx = 10  # À adapter
feature_names = ["Volume", "Occupation", "Niveau de congestion", "Vitesse moyenne"]

# Création de la figure
fig, axs = plt.subplots(2, 2, figsize=(15, 12))
axs = axs.flatten()
colors = ['blue', 'green', 'purple', 'brown']

# Tracer chaque caractéristique
for feature_idx in range(4):
    ax = axs[feature_idx]
    
    # Extraction des données
    true_day_values = true_values[day_indices, node_idx, feature_idx]
    pred_day_values = predictions[day_indices, node_idx, feature_idx]
    
    # Calcul des métriques pour ce jour/capteur/feature
    mae = mean_absolute_error(true_day_values, pred_day_values)
    rmse = np.sqrt(mean_squared_error(true_day_values, pred_day_values))
    r2 = r2_score(true_day_values, pred_day_values)
    
    # Tracé
    day_dates = [dates[i] for i in day_indices]
    ax.plot(day_dates, true_day_values, label="Valeurs réelles", 
            linewidth=2, color=colors[feature_idx])
    ax.plot(day_dates, pred_day_values, label="Prédictions", 
            linewidth=2, linestyle='--', color=colors[feature_idx])
    
    # Configuration des axes
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend()
    ax.set_xlabel("Heure")
    ax.set_ylabel(feature_names[feature_idx])
    ax.set_title(f"{feature_names[feature_idx]} - Capteur {node_idx}\n"
                f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.2f}")

# Titre global
plt.suptitle(f"Prédictions vs Valeurs réelles - {specific_day.strftime('%d/%m/%Y')}", 
             fontsize=16, fontweight='bold')

# Ajustement et sauvegarde
plt.tight_layout()
plt.savefig(f"prediction_sensor_{node_idx}_day_{specific_day.strftime('%Y%m%d')}.png", 
            dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Récupérer la matrice d'adjacence après entraînement de MTGNN
# Après entraînement
adj = model.get_adjacency_matrix()
print("Matrice shape:", adj.shape)
print(adj)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau # Importation de l'ordonnanceur
from torch.utils.data import DataLoader, Dataset
from torch_geometric.nn import GATConv # Remplacement de GCNConv par GATConv
from sklearn.preprocessing import StandardScaler # Remplacement de MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import BallTree
import math
import matplotlib.pyplot as plt
import time
import copy



# Paramètres du modèle et de l'entraînement (ajustés/ajoutés)
num_nodes = 421
NUM_FEATURES = 11
SEQ_LEN = 12
HORIZON = 1
BATCH_SIZE = 32
EPOCHS = 20 # Augmenter potentiellement, l'early stopping gère la fin
LEARNING_RATE = 0.001 # Point de départ, à tuner !
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
HIDDEN_DIM = 64
GAT_LAYERS = 2      # Nombre de couches GAT
GAT_HEADS = 4       # Nombre de têtes d'attention pour GATConv (output dim = HIDDEN_DIM * GAT_HEADS si concat, ou HIDDEN_DIM si moyenné)
GRU_LAYERS = 1
DROPOUT_RATE = 0.3
num_targets=4

# Early Stopping
EARLY_STOPPING_PATIENCE = 5 # Légèrement augmenté
EARLY_STOPPING_DELTA = 0.0

# Paramètres pour la construction du graphe
DISTANCE_THRESHOLD_KM = 1.0
ADD_SELF_LOOPS = True # Important pour GAT aussi
EDGE_WEIGHT_SIGMA = 0.5 # Conservé pour info, mais GAT apprendra ses propres poids

print(f"Using device: {DEVICE}")



def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    lat1, lat2 = map(math.radians, [lat1, lat2])
    a = math.sin(dLat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dLon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def build_spatial_graph_for_gat(sensor_locations_df, id_col, lat_col, lon_col, distance_threshold_km, add_self_loops=True):
    """
    Construit l'index des arêtes pour GAT (basé sur la proximité).
    Retourne adj_matrix (binaire ou pondérée pour info) et edge_index.
    """
    locations = sensor_locations_df[[id_col, lat_col, lon_col]].drop_duplicates(subset=[id_col]).set_index(id_col)
    sensor_ids = locations.index.tolist()
    num_nodes = len(sensor_ids)
    sensor_id_to_index = {sensor_id: i for i, sensor_id in enumerate(sensor_ids)}

    print(f"Building graph connectivity for {num_nodes} sensors...")
    coords_rad = np.radians(locations[[lat_col, lon_col]].values)
    tree = BallTree(coords_rad, metric='haversine')
    dist_threshold_rad = distance_threshold_km / 6371

    adj_matrix = np.zeros((num_nodes, num_nodes), dtype=np.float32) # Pour info ou debug
    edge_list = []

    # Correction ici: supprimer le '[0]' à la fin car return_distance=False par défaut
    indices = tree.query_radius(coords_rad, r=dist_threshold_rad)

    # Maintenant 'indices' est un tableau où chaque élément est un tableau d'indices de voisins
    for i, neighbors_indices in enumerate(indices):
        # neighbors_indices est maintenant un tableau (itérable)
        for j in neighbors_indices:
            if i == j: continue
            adj_matrix[i, j] = 1 # Matrice binaire simple
            adj_matrix[j, i] = 1 # Symétrique
            # Ajouter la paire (i,j) si elle n'est pas déjà dans edge_list pour éviter doublons GAT
            # (Bien que set() à la fin gère cela aussi)
            if (i,j) not in edge_list and (j,i) not in edge_list:
                 edge_list.append((i, j))


    # Gérer les self-loops pour edge_index si GATConv ne le fait pas (souvent mieux de les inclure)
    if add_self_loops:
        np.fill_diagonal(adj_matrix, 1.0)
        for i in range(num_nodes):
             # Ajouter (i,i) à edge_list si pas déjà présent (pour être sûr)
             if not any(item == (i, i) for item in edge_list):
                 edge_list.append((i, i))

    # Convertir en tensor (set() n'est plus nécessaire si on gère les doublons plus haut)
    # S'assurer qu'il n'y a pas de doublons si on n'a pas géré plus haut
    # edge_list = list(set(edge_list)) # Optionnel si la logique ci-dessus est bonne
   
    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()

    print(f"Graph connectivity built with {edge_index.shape[1]} edges (including self-loops if added).")
    return adj_matrix, edge_index, sensor_id_to_index



In [ ]:

# -- Graph Construction --
adj_matrix, edge_index, sensor_id_to_index = build_spatial_graph_for_gat(
    unique_combinations, 'id','lon','lat', DISTANCE_THRESHOLD_KM, ADD_SELF_LOOPS
)

# Move edge_index to the correct device
edge_index = edge_index.to(DEVICE) 
data_tensor = torch.tensor(data_restructured_final, dtype=torch.float32)

def create_sequences(data, SEQ_LEN, horizon):
    sequences = []
    targets = []
    for i in range(len(data) - SEQ_LEN - horizon):
        seq = data[i:i+SEQ_LEN]  # (seq_len, num_nodes, num_features)
        target = data[i+SEQ_LEN, :, :4]  # (num_nodes, 4)
        sequences.append(seq)
        targets.append(target)
    return torch.stack(sequences), torch.stack(targets)

sequences, targets = create_sequences(data_tensor, SEQ_LEN, HORIZON)

def temporal_split(sequences, targets, train_ratio=0.8, val_ratio=0.1):
    num_samples = len(sequences)
    train_end = int(train_ratio * num_samples)
    val_end = train_end + int(val_ratio * num_samples)
    
    # Découpage séquentiel
    train_seq, train_targ = sequences[:train_end], targets[:train_end]
    val_seq, val_targ = sequences[train_end:val_end], targets[train_end:val_end]
    test_seq, test_targ = sequences[val_end:], targets[val_end:]
    
    return train_seq, val_seq, test_seq, train_targ, val_targ, test_targ

# Application
train_seq, val_seq, test_seq, train_targ, val_targ, test_targ = temporal_split(sequences, targets)

# -------------------------------------------------
# 2. Dataset et DataLoader
# -------------------------------------------------
class TrafficDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

# Création des datasets
train_dataset = TrafficDataset(train_seq, train_targ)
val_dataset = TrafficDataset(val_seq, val_targ)
test_dataset = TrafficDataset(test_seq, test_targ)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,num_workers=4, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=4, persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=4, persistent_workers=True)




In [ ]:
# -- 6. Définition du Modèle ST-GNN (avec GAT) --
class STGNN_GAT(nn.Module):
    def __init__(self, num_nodes, num_features, num_targets, seq_len, horizon, hidden_dim, gat_layers=1, gat_heads=1, gru_layers=1, dropout_rate=0.0):
        super(STGNN_GAT, self).__init__()
        self.num_nodes = num_nodes
        self.num_targets = num_targets
        self.horizon = horizon
        self.hidden_dim = hidden_dim
        self.gat_heads = gat_heads

        # Couche GRU initiale
        self.gru1 = nn.GRU(num_features, hidden_dim, gru_layers, batch_first=True)
        self.ln1 = nn.LayerNorm([num_nodes, hidden_dim])

        # Couches GAT
        self.gat_layers = nn.ModuleList()
        self.gat_lns = nn.ModuleList()
        # La première couche GAT prend hidden_dim en entrée
        self.gat_layers.append(GATConv(hidden_dim, hidden_dim, heads=gat_heads, dropout=dropout_rate))
        self.gat_lns.append(nn.LayerNorm([num_nodes, hidden_dim * gat_heads])) # La sortie est multipliée par le nb de têtes
       
        # Les couches GAT suivantes prennent hidden_dim * gat_heads en entrée
        for _ in range(1, gat_layers):
            self.gat_layers.append(GATConv(hidden_dim * gat_heads, hidden_dim, heads=gat_heads, dropout=dropout_rate))
            self.gat_lns.append(nn.LayerNorm([num_nodes, hidden_dim * gat_heads]))

        self.relu = nn.ELU() # ELU est souvent utilisé avec GAT
        self.dropout = nn.Dropout(p=dropout_rate)

        # Couche de sortie : l'entrée dépend de la sortie de la dernière couche GAT
        # Si on moyenne les têtes sur la dernière couche, l'entrée est hidden_dim
        # Si on concatène, c'est hidden_dim * gat_heads. Simplifions en moyennant implicitement ou via une couche linéaire.
        # Ou ajustons la dernière couche GAT pour sortir hidden_dim directement en moyennant.
        # Ici, on ajoute une couche linéaire pour mapper la sortie GAT (concaténée) vers la dim de sortie voulue.
        # Si la dernière couche GAT moyenne les têtes, utiliser : self.fc = nn.Linear(hidden_dim, horizon * num_targets)
        self.fc = nn.Linear(hidden_dim * gat_heads, horizon * num_targets) # Si la sortie GAT est concaténée

    def forward(self, x, edge_index): # Ne prend plus edge_weight
        # x shape: [batch_size, seq_len, num_nodes, num_features]
        batch_size, seq_len, _, _ = x.shape

        # Traitement temporel initial
        x_reshaped = x.permute(0, 2, 1, 3).reshape(batch_size * self.num_nodes, seq_len, -1)
        out, _ = self.gru1(x_reshaped)
        gru1_out = out[:, -1, :].reshape(batch_size, self.num_nodes, self.hidden_dim)
        gru1_out_norm = self.ln1(gru1_out)

        # Traitement spatial avec GAT
        gat_input = gru1_out_norm # [batch_size, num_nodes, hidden_dim]
       
        hidden_gat_list = []
        # Itérer sur le batch car GATConv standard traite un graphe à la fois
        for i in range(batch_size):
            hidden_batch = gat_input[i] # [num_nodes, hidden_dim]
            for k, layer in enumerate(self.gat_layers):
                # GATConv attend (x, edge_index)
                hidden_batch = self.relu(layer(hidden_batch, edge_index))
                # Appliquer LayerNorm après chaque GAT
                hidden_batch = self.gat_lns[k](hidden_batch)
            hidden_gat_list.append(hidden_batch)
       
        final_features = torch.stack(hidden_gat_list) # [batch_size, num_nodes, hidden_dim * gat_heads]

        final_features_dropout = self.dropout(final_features)

        # Couche de sortie
        output = self.fc(final_features_dropout) # [batch_size, num_nodes, horizon * num_targets]

        return output


# Instanciation du modèle
model2 = STGNN_GAT(num_nodes=num_nodes,
                  num_features=num_features,
                  num_targets=num_targets,
                  seq_len=SEQ_LEN,
                  horizon=HORIZON,
                  hidden_dim=HIDDEN_DIM,
                  gat_layers=GAT_LAYERS,
                  gat_heads=GAT_HEADS,
                  gru_layers=GRU_LAYERS,
                  dropout_rate=DROPOUT_RATE).to(DEVICE)

print("Model Architecture:")
print(model2)
print(f"Total Parameters: {sum(p.numel() for p in model2.parameters() if p.requires_grad)}")

# -- 7. Entraînement du Modèle (avec HuberLoss et LR Scheduler) --
criterion = nn.HuberLoss() # Changement de fonction de perte
optimizer = optim.Adam(model2.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True) # Ordonnanceur

print("Starting training with HuberLoss, LR Scheduler, and Early Stopping...")
train_losses = []
val_losses = []
best_val_loss = float('inf')
epochs_no_improve = 0
best_model_state = None

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    model2.train()
    running_train_loss = 0.0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(DEVICE), batch_y.to(DEVICE)
        optimizer.zero_grad()
        outputs = model2(batch_x, edge_index) # Ne passe plus edge_weight
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
    avg_train_loss = running_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    model2.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for batch_x_val, batch_y_val in val_loader:
            batch_x_val, batch_y_val = batch_x_val.to(DEVICE), batch_y_val.to(DEVICE)
            outputs_val = model2(batch_x_val, edge_index) # Ne passe plus edge_weight
            loss_val = criterion(outputs_val, batch_y_val)
            running_val_loss += loss_val.item()
    avg_val_loss = running_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    epoch_end_time = time.time()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}, LR: {current_lr:.6f}, Time: {epoch_end_time - epoch_start_time:.2f}s")

    # Mise à jour de l'ordonnanceur basé sur la perte de validation
    scheduler.step(avg_val_loss)

    # Early Stopping Check
    if avg_val_loss < best_val_loss - EARLY_STOPPING_DELTA:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        best_model_state = copy.deepcopy(model2.state_dict())
        print(f"  Validation loss improved to {best_val_loss:.6f}. Saving model state.")
    else:
        epochs_no_improve += 1
        print(f"  Validation loss did not improve for {epochs_no_improve} epoch(s).")

    if epochs_no_improve >= EARLY_STOPPING_PATIENCE:
        print(f"\nEarly stopping triggered after {epoch + 1} epochs.")
        break

# Charger le meilleur modèle
if best_model_state:
    print("\nLoading best model state found during training.")
    model2.load_state_dict(best_model_state)
else:
     print("\nWarning: No best model state saved. Using last state.")

print("Training finished.")

In [ ]:
# -- 8. Évaluation sur l'Ensemble de Test --
print("Evaluating on test set using the best model...")
model2.eval()  # On utilise model2 partout maintenant

# Déplacer edge_index sur le bon device
edge_index = edge_index.to(DEVICE)

predictions = []
actuals = []

with torch.no_grad():
    for batch_x_test, batch_y_test in test_loader:
        batch_x_test, batch_y_test = batch_x_test.to(DEVICE), batch_y_test.to(DEVICE)
        outputs_test = model2(batch_x_test, edge_index)  # Changé model -> model2
        
        # Conversion en numpy et stockage
        predictions.append(outputs_test.cpu().numpy())
        actuals.append(batch_y_test.cpu().numpy())

# Concaténation des résultats
predictions = np.concatenate(predictions, axis=0)
actuals = np.concatenate(actuals, axis=0)

# Redimensionnement pour conserver la structure [samples, nodes, horizon, targets]
predictions_final = predictions.reshape(-1, num_nodes, HORIZON, num_targets)
actuals_final = actuals.reshape(-1, num_nodes, HORIZON, num_targets)

# Calcul des métriques
print("\n--- Test Set Evaluation Metrics (Standardized Scale) ---")
TARGET_COLS = ['volume', 'occupation', 'congestion_level', 'speed_avg']  # Assurez-vous que c'est défini

for i, target_name in enumerate(TARGET_COLS):
    pred = predictions_final[:, :, 0, i].flatten()
    true = actuals_final[:, :, 0, i].flatten()
    
    # Filtrage des valeurs non finies
    mask = np.isfinite(true) & np.isfinite(pred)
    true = true[mask]
    pred = pred[mask]
    
    if len(true) < 2:
        print(f"{target_name}: Not enough valid samples")
        continue
    
    # Calcul des métriques
    mse = mean_squared_error(true, pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(true - pred))
    r2 = r2_score(true, pred)
    
    print(f"{target_name}:")
    print(f"  MSE: {mse:.4f}, RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}, R²: {r2:.4f}")
    print("-"*40)

# -- 9. Visualisation ( étendue à toutes les cibles) --
print("Generating visualizations for all target variables...")

# 1. Courbes d'apprentissage (Loss)
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
best_epoch_marker = len(val_losses) - epochs_no_improve - 1 if best_model_state and epochs_no_improve < len(val_losses) else len(val_losses) - 1
plt.axvline(x=best_epoch_marker, color='r', linestyle='--', label=f'Best Epoch ({best_epoch_marker+1})')
plt.title('Model2 Training and Validation Loss (GAT Version)')  # Mise à jour du titre
plt.xlabel('Epochs')
plt.ylabel('Huber Loss')
plt.legend()
plt.grid(True)
plt.savefig('training_validation_loss_gat.png')
print("Saved: training_validation_loss_gat.png")
plt.close()

In [ ]:
# Choisir un capteur à visualiser (peut être modifié)
sensor_index_to_plot = 0
sensor_id_plotted = unique_combinations['id'][sensor_index_to_plot]
print(f"\nGenerating comparison plots for Sensor ID: {sensor_id_plotted} (Index: {sensor_index_to_plot})")

for i, target_name in enumerate(TARGET_COLS):
    print(f"  Generating plots for target: {target_name}")

    # 2. Comparaison Actuel vs. Prédit (Série Temporelle) pour ce capteur et cette cible
    actual_sensor = actuals_final[:, sensor_index_to_plot, 0, i]
    predicted_sensor = predictions_final[:, sensor_index_to_plot, 0, i]

    plt.figure(figsize=(15, 6))
    plt.plot(actual_sensor, label='Actual', color='blue', marker='.', markersize=4, linestyle='')
    plt.plot(predicted_sensor, label='Predicted', color='red', alpha=0.7)
    plt.title(f'Actual vs Predicted {target_name} for Sensor {sensor_id_plotted} (Test Set - GAT Model)')
    plt.xlabel('Time Steps (in test set)')
    plt.ylabel(f'{target_name}') # L'échelle est celle des données originales grâce à inverse_transform
    plt.legend()
    plt.grid(True)
    filename_ts = f'actual_vs_predicted_{target_name}_sensor_{sensor_id_plotted}_gat.png'
    plt.savefig(filename_ts)
    print(f"    Saved: {filename_ts}")
    # plt.show() # Décommentez pour afficher chaque graphique immédiatement
    plt.close() # Ferme la figure

    # 3. Scatter plot Prédit vs Actuel pour cette cible (tous les nœuds)
    plt.figure(figsize=(8, 8))
    actual_flat = actuals_final[:, :, 0, i].flatten()
    pred_flat = predictions_final[:, :, 0, i].flatten()
    mask = np.isfinite(actual_flat) & np.isfinite(pred_flat) # Utiliser seulement les points finis

    if np.sum(mask) > 0: # Vérifier s'il y a des points valides à tracer
        plt.scatter(actual_flat[mask], pred_flat[mask], alpha=0.2, s=5)
        plt.xlabel(f'Actual {target_name}')
        plt.ylabel(f'Predicted {target_name}')
        plt.title(f'Predicted vs Actual {target_name} (All nodes, Test Set - GAT Model)')
        # Ligne y=x pour référence
        min_val = min(np.min(actual_flat[mask]), np.min(pred_flat[mask]))
        max_val = max(np.max(actual_flat[mask]), np.max(pred_flat[mask]))
        plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--')
        plt.grid(True)
        plt.axis('equal') # Assure une échelle égale sur les deux axes
        filename_scatter = f'scatter_predicted_vs_actual_{target_name}_gat.png'
        plt.savefig(filename_scatter)
        print(f"    Saved: {filename_scatter}")
    else:
        print(f"    Skipped scatter plot for {target_name}: No valid data points.")

    # plt.show() # Décommentez pour afficher chaque graphique immédiatement
    plt.close() # Ferme la figure

print("\nAll visualizations generated and saved as PNG files.")
print("Code execution finished.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime, timedelta

# Utilisez les tableaux redimensionnés (4D) pour la visualisation
predictions = predictions_final
actuals = actuals_final

# Définir le jour spécifique à visualiser
specific_day = datetime(2021, 9, 2)  # Adaptez à votre cas

# Créer les dates
start_date = datetime(2021, 6, 1)  # Date de début de vos données
dates = [start_date + timedelta(hours=2*i) for i in range(actuals.shape[0])]

# Filtrer les indices du jour spécifique
day_indices = [i for i, date in enumerate(dates) if date.date() == specific_day.date()]

# Choisir un nœud spécifique
node_idx = 10 # À adapter

# Noms des caractéristiques
feature_names = ["Volume", "Occupation", "Niveau de congestion", "Vitesse moyenne"]

# Création de la figure
fig, axs = plt.subplots(2, 2, figsize=(15, 12))
axs = axs.flatten()

# Couleurs pour les courbes
colors = ['blue', 'green', 'purple', 'brown']

# Tracer chaque caractéristique
for feature_idx in range(4):
    ax = axs[feature_idx]
    
    # Extraction des données depuis les tableaux 4D
    true_day_values = actuals[day_indices, node_idx, 0, feature_idx]  # [samples, nodes, horizon, features]
    pred_day_values = predictions[day_indices, node_idx, 0, feature_idx]
    
    # Tracé
    ax.plot(dates[day_indices[0]:day_indices[-1]+1], true_day_values, 
            label="Valeurs réelles", linewidth=2, color=colors[feature_idx])
    ax.plot(dates[day_indices[0]:day_indices[-1]+1], pred_day_values, 
            label="Prédictions", linewidth=2, linestyle='--', color=colors[feature_idx])
    
    # Configuration des axes
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend()
    ax.set_xlabel("Heure")
    ax.set_ylabel(feature_names[feature_idx])
    ax.set_title(f"{feature_names[feature_idx]} - Capteur {node_idx}")

# Titre global
plt.suptitle(f"Prédictions vs Valeurs réelles - {specific_day.strftime('%d/%m/%Y')}", 
             fontsize=16, fontweight='bold')

# Ajustement et sauvegarde
plt.tight_layout()
plt.savefig(f"prediction_sensor_{node_idx}_day_{specific_day.strftime('%Y%m%d')}.png", 
            dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import numpy as np

# Supposons que vous avez ces données (à adapter selon vos besoins)
# true_values: valeurs réelles pour un nœud et une feature spécifique
# predictions_final, predictions, predictions1: prédictions des 3 modèles

# Créer des dates pour l'axe x (exemple pour 24 heures avec un pas de 2h)
dates = [datetime(2021, 6, 1) + timedelta(hours=2 * i) for i in range(12)]  # 12 points pour 24h

# Sélectionner un nœud et une feature spécifique
node_idx = 0  # Premier capteur
feature_idx = 0  # Première caractéristique (par exemple volume)

# Convertir les prédictions en numpy array si ce ne sont pas déjà des arrays
predictions_final = np.array(predictions_final)
predictions = np.array(predictions)
predictions1 = np.array(predictions1)

# Extraire les prédictions pour le nœud et la feature sélectionnés
pred_model_1 = predictions_final[:, node_idx, feature_idx]  # STGNN
pred_model_2 = predictions[:, node_idx, feature_idx]        # MTGNN
pred_model_3 = predictions1[:, node_idx, feature_idx]       # ASTGCN

# Créer une figure
plt.figure(figsize=(12, 6))

# Tracer les valeurs réelles
plt.plot(dates, true_values, label="Valeurs réelles", linewidth=2, color="black")

# Tracer les prédictions des 3 modèles
plt.plot(dates, pred_model_1, label="STGNN", linestyle="--", color="blue")
plt.plot(dates, pred_model_2, label="MTGNN", linestyle="--", color="green")
plt.plot(dates, pred_model_3, label="ASTGCN", linestyle="--", color="red")

# Formater l'axe des x pour afficher les heures
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=2))

# Ajouter légendes, titres, etc.
plt.xlabel("Heure", fontsize=12)
plt.ylabel("Valeur (Volume, Vitesse, etc.)", fontsize=12)
plt.title(f"Comparaison des prédictions pour le capteur {node_idx} - Caractéristique {feature_idx}", fontsize=14)
plt.legend(loc='upper right')

# Ajouter une grille
plt.grid(True, linestyle='--', alpha=0.7)

# Ajuster la mise en page
plt.tight_layout()

# Afficher le graphique
plt.show()